# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm, metrics
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer, r2_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import time

# 1. Import Data

In [3]:
import pandas as pd

df_qt = pd.read_csv('./data/qt-public.csv')
df_tbtl = pd.read_excel('./data/tbtl-public.ods', engine='odf')
df_th = pd.read_csv('./data/th-public.csv')
df_ck = pd.read_csv('./data/ck-public.csv')
df = pd.read_csv("./data/annonimized.csv")

In [4]:
print(df_qt.sample(5))
print(df_tbtl.sample(5))
print(df_th.sample(5))
print(df_ck.sample(5))
print(df.sample(5))
print(df.info())

                                         hash diemqt
556  60c257442a0ef9aa6eb039f58c40be980bfe58b7     10
54   0821ac06ae77101d7dd9b8dd103f79ffa46657ab      7
582  648b6d4b1c95ee18825cb3cdd69c033a10f3633e      9
759  8460eaaf887a6289fb156f7562fb739ba8e9629e     10
542  5e6c70b4de90c2415359c22c351d8b827d22886c      8
                                     username  TBTL
302  32991cd9c0115e4bd4387c8fa3ed559f4eba345e  7.92
564  5e092e334855a0d8739d53c39798bb26f3376959  8.71
100  0fbef055915162cb6d6501558499544d6dedb799  8.60
356  3a3cf426569ff3ed1638fefb8d0a7bca73e0bdb7  7.34
63   09cbe188b9d1e965edf6c1c46a96b9306ec2f329  7.83
                                         hash   TH
175  1db8dc5d1005cd6d742fd9bfba759aeb2babab66    8
60   09cbbae64d03af8ca53224aaef7657171ba27f50    9
633  6e849b2924dd2fc0a7efc8ef0a5f614ce2409b4f    5
701  7953012ece7e7f5c184ec3ce99de925dc01c46ef  9.5
118  12d6e24985ac80a581968fb1f36d699e6dd1cd33    7
                                         hash   CK
298  3399fb9c

# 2. Preprocessing

In [5]:
#Taọ dataframe copy để dễ xử lý
df_cp = df.copy()

# Đổi tên các cột để dễ sử dụng
df_cp['problem_id'] = df["concat('it001',`problem_id`)"]
df_cp['username'] = df["concat('it001', username)"]
df_cp['assignment_id'] = df["concat('it001',`assignment_id`)"]
df_cp['language_id'] = df["concat('it001',`language_id`)"]

In [6]:
df_cp['language_id']

0         it0012
1         it0012
2         it0012
3         it0012
4         it0012
           ...  
295193    it0012
295194    it0012
295195    it0012
295196    it0012
295197    it0012
Name: language_id, Length: 295198, dtype: object

In [7]:
df_cp['created_at'] = pd.to_datetime('2020-' + df_cp['created_at'], format='%Y-%m-%d %H:%M:%S')
df_cp['updated_at'] = pd.to_datetime('2020-' + df_cp['updated_at'], format='%Y-%m-%d %H:%M:%S')

In [8]:
df_cp.loc[df_cp['created_at'].dt.month >= 8, 'created_at'] -= pd.DateOffset(years=1)
df_cp.loc[df_cp['updated_at'].dt.month >= 8, 'updated_at'] -= pd.DateOffset(years=1)

In [9]:
df_cp['created_at'].describe()

count                           295198
mean     2019-11-16 14:10:00.088994304
min                2019-09-05 02:01:26
25%                2019-10-11 04:58:21
50%                2019-11-07 10:54:23
75%      2019-12-26 06:58:59.249999872
max                2020-05-21 04:27:38
Name: created_at, dtype: object

In [10]:
# Trích xuất số tuần và ngày trong tuần từ 'updated_at'
df_cp['week_number'] = df_cp['updated_at'].dt.isocalendar().week
df_cp['day_of_week'] = df_cp['updated_at'].dt.dayofweek

In [11]:
print(df_tbtl.dtypes)
print(df_th.dtypes)
print(df_qt.dtypes)
print(df_ck.dtypes)

username     object
TBTL        float64
dtype: object
hash    object
TH      object
dtype: object
hash      object
diemqt    object
dtype: object
hash     object
CK      float64
dtype: object


In [12]:
df_th['TH'] = pd.to_numeric(df_th['TH'], errors='coerce')
df_qt['diemqt'] = pd.to_numeric(df_qt['diemqt'], errors='coerce')

In [13]:
df_cp = df_cp.merge(df_tbtl, on='username', how='left')
df_cp = df_cp.merge(df_th, left_on='username', right_on='hash', how='left')
df_cp = df_cp.rename(columns={'hash': 'hash_th'})
df_cp = df_cp.merge(df_qt, left_on='username', right_on='hash', how='left')
df_cp = df_cp.rename(columns={'hash': 'hash_qt'})
df_cp = df_cp.merge(df_ck, left_on='username', right_on='hash', how='left')
df_cp = df_cp.rename(columns={'hash': 'hash_ck'})

In [14]:
df_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295198 entries, 0 to 295197
Data columns (total 24 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   concat('it001',`assignment_id`)  295198 non-null  object        
 1   concat('it001',`problem_id`)     295198 non-null  object        
 2   concat('it001', username)        295198 non-null  object        
 3   is_final                         295198 non-null  int64         
 4   status                           295198 non-null  object        
 5   pre_score                        295198 non-null  int64         
 6   coefficient                      295198 non-null  int64         
 7   concat('it001',`language_id`)    295198 non-null  object        
 8   created_at                       295198 non-null  datetime64[ns]
 9   updated_at                       295198 non-null  datetime64[ns]
 10  judgement                        295198 non-

In [15]:
df_cp.isna().sum()

concat('it001',`assignment_id`)         0
concat('it001',`problem_id`)            0
concat('it001', username)               0
is_final                                0
status                                  0
pre_score                               0
coefficient                             0
concat('it001',`language_id`)           0
created_at                              0
updated_at                              0
judgement                               0
problem_id                              0
username                                0
assignment_id                           0
language_id                             0
week_number                             0
day_of_week                             0
TBTL                               133802
hash_th                            138158
TH                                 138356
hash_qt                            138158
diemqt                             138294
hash_ck                            138158
CK                                

In [16]:
def encode_assignment_problem(df, target, alpha=5):
    global_mean = df[target].mean()
    
    count_assignment = df_cp.groupby('assignment_id')[target].count()
    count_problem = df_cp.groupby('problem_id')[target].count()

    map_assignment = (df_cp.groupby('assignment_id')[target].sum() + (alpha * global_mean)) / (count_assignment + alpha)
    map_problem = (df_cp.groupby('problem_id')[target].sum() + (alpha * global_mean)) / (count_problem + alpha)

    map_assignment.fillna(global_mean, inplace=True)
    map_problem.fillna(global_mean, inplace=True)

    assignment_encode = df_cp['assignment_id'].map(map_assignment)
    problem_encode = df_cp['problem_id'].map(map_problem)

    return pd.DataFrame({'assignment_encode': assignment_encode, 'problem_encode': problem_encode})

df_cp[['assignment_encode_tbtl', 'problem_encode_tbtl']] = encode_assignment_problem(df_cp, 'TBTL')
df_cp[['assignment_encode_th', 'problem_encode_th']] = encode_assignment_problem(df_cp, 'TH')
df_cp[['assignment_encode_qt', 'problem_encode_qt']] = encode_assignment_problem(df_cp, 'diemqt')
df_cp[['assignment_encode_ck', 'problem_encode_ck']] = encode_assignment_problem(df_cp, 'CK')

In [17]:
df_cp.isna().sum()

concat('it001',`assignment_id`)         0
concat('it001',`problem_id`)            0
concat('it001', username)               0
is_final                                0
status                                  0
pre_score                               0
coefficient                             0
concat('it001',`language_id`)           0
created_at                              0
updated_at                              0
judgement                               0
problem_id                              0
username                                0
assignment_id                           0
language_id                             0
week_number                             0
day_of_week                             0
TBTL                               133802
hash_th                            138158
TH                                 138356
hash_qt                            138158
diemqt                             138294
hash_ck                            138158
CK                                

In [18]:
df_cp.head()

,"concat('it001',`assignment_id`)","concat('it001',`problem_id`)","concat('it001', username)",is_final,status,pre_score,coefficient,"concat('it001',`language_id`)",created_at,updated_at,...,hash_ck,CK,assignment_encode_tbtl,problem_encode_tbtl,assignment_encode_th,problem_encode_th,assignment_encode_qt,problem_encode_qt,assignment_encode_ck,problem_encode_ck
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2019-10-09 08:02:04,2019-10-09 08:06:58,...,NaN,NaN,7.750261,7.784867,6.701809,7.407447,5.888731,8.080647,4.418246,5.258739
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2019-10-09 08:04:41,2019-10-09 08:04:51,...,NaN,NaN,7.750261,7.784867,6.701809,7.407447,5.888731,8.080647,4.418246,5.258739
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2019-10-09 08:06:49,2019-10-09 08:06:58,...,NaN,NaN,7.750261,7.784867,6.701809,7.407447,5.888731,8.080647,4.418246,5.258739
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2019-10-09 08:47:52,2019-10-09 08:48:01,...,NaN,NaN,7.750261,7.687121,6.701809,7.066149,5.888731,7.976060,4.418246,4.991403
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2019-10-09 09:19:35,2019-10-09 09:19:45,...,NaN,NaN,7.750261,7.728919,6.701809,7.079039,5.888731,8.077367,4.418246,4.831182


In [19]:
map_problem_max_prescore = df_cp.groupby('problem_id')['pre_score'].max()
df_cp['max_pre_score'] = df_cp['problem_id'].map(map_problem_max_prescore)

In [20]:
df_cp['max_pre_score']

0         10000
1         10000
2         10000
3         10000
4         10000
          ...  
295193    10000
295194    10000
295195    10000
295196    10000
295197    10000
Name: max_pre_score, Length: 295198, dtype: int64

# 3. Feature Engineering

### 3.1 Thống kê tổng problems, assignments

In [21]:
def get_total_problems_assignments(df: pd.DataFrame) -> dict:
    """ Hàm thống kê tổng số problems, tổng số assignments

    Args:
        df (pd.DataFrame): Assignment DataFrame

    Returns:
        dict: Series Total problems, Total assignments 
    """

    total_problems = df.groupby('username')['problem_id'].nunique()
    total_assignments = df.groupby('username')['assignment_id'].nunique()

    return {'total_problems' : total_problems, 
            'total_assignments' : total_assignments}


### 3.2 Thống kê tỉ lệ nộp bài

In [22]:
def get_rate_submission_correct(df: pd.DataFrame) -> dict:
    """Hàm thống kê tỉ lệ làm bài đúng sớm, đúng trễ, chưa hoàn thành

    Args:
        df (pd.DataFrame): Assignment DataFrame

    Returns:
        dict: Series Rate early correct, Rate late correct, not completed
    """    

    total_problems = get_total_problems_assignments(df)['total_problems']
    
    early_correct_rate = df[(df['pre_score'] == 10000) & (df['coefficient'] == 100)].groupby('username')['problem_id'].nunique() / total_problems
    early_correct_rate.fillna(0, inplace=True)


    not_completed_rate = 1 - early_correct_rate

    return {'early_correct_rate': early_correct_rate,
            'not_completed_rate': not_completed_rate}


### 3.3 Thống kê tỉ lệ nộp bài thành công, thất bại

In [23]:
def get_successful_unsuccessful_submissions_rate(df: pd.DataFrame) -> dict:
    """Thống kê tỉ lệ nộp bài thành công

    Args:
        df (pd.DataFrame): Assignment DataFrame

    Returns:
        dict: Series Successful, unsuccessful submission avg
    """

    successful_submissions = df[df['pre_score'] == 10000].groupby('username').size()
    unsuccessful_submissions = df[df['pre_score'] < 10000].groupby('username').size()
    successful_rate_avg = successful_submissions / (successful_submissions + unsuccessful_submissions)
    unsuccessful_rate_avg = unsuccessful_submissions / (successful_submissions + unsuccessful_submissions)

    successful_rate_avg.fillna(0, inplace=True)
    unsuccessful_rate_avg.fillna(0, inplace=True)

    return {'successful_rate_avg': successful_rate_avg,
            'unsuccessful_rate_avg': unsuccessful_rate_avg}

### 3.4 Thống kê tỉ lệ lỗi 

In [24]:
def get_error_info(df: pd.DataFrame) -> dict:
    """Thống kê số lượng và tỉ lệ các lõi của submission

    Args:
        df (pd.DataFrame): Assignment DataFrame

    Returns:
        dict: Dict: Serise group err: Limit_Exceeded, Run_Time_Error, Wrong, Banned
    """

    # Các nhóm loại lỗi
    err_groups = {
        "Limit_Exceeded": ["Time Limit Exceeded", "Memory Limit Exceeded"],
        "Run_Time_Error": ["was not declared in this scope", "invalid type", "expected declaration"],
        "Wrong": ["WRONG"],
        "Banned": ["is banned"]
    }
     
    err_count = dict()

    total_assignments = get_total_problems_assignments(df)['total_assignments']

    # Tính toán số lượng đánh giá
    for err_group, err_strs in err_groups.items():
        err_count[err_group] = df['judgement'].apply(lambda judge: 
                                                        any(err_str in judge for err_str in err_strs)).groupby(df['username']).sum()
        err_count[err_group + '_avg'] = err_count[err_group] / total_assignments

    return err_count
    

### 3.5 Thống kê bài tập khó, dễ, trung bình

In [25]:
def get_difficulty_mapping(df: pd.DataFrame, easy_per: float = 0.7, difficult_per: float = 0.3)-> dict:
    """Tạo mapping độ khó dễ trung bình cho problem dựa voà số sinh viên hoàn thành problem

    Args:
        df (pd.DataFrame): Assignment DataFrame
    Returns:
        dict: dict map problem to difficulty
    """

    # Tính toán tỉ lệ hoàn thành bài
    completed_problems = df_cp[df_cp['pre_score'] == 10000].groupby('problem_id')['username'].nunique().reset_index()
    completed_problems.columns = ['problem_id', 'num_completed']

    # Tính tổng số lượng người dùng đã nộp bài cho từng bài
    total_submissions = df_cp.groupby('problem_id')['username'].nunique().reset_index()
    total_submissions.columns = ['problem_id', 'num_submissions']

    # Tính toán tỉ lệ hoàn thành bài và điền giá trị NaN bằng 0
    problem_completion_rate = pd.merge(total_submissions, completed_problems, on='problem_id', how='left')
    problem_completion_rate['completion_rate'] = problem_completion_rate['num_completed'] / problem_completion_rate['num_submissions']
    problem_completion_rate['completion_rate'] = problem_completion_rate['completion_rate'].fillna(0)

    # Xác định ngưỡng cho các bài dễ, trung bình và khó
    thresholds = problem_completion_rate['completion_rate'].quantile([difficult_per, easy_per])
    problem_completion_rate['difficulty'] = pd.cut(problem_completion_rate['completion_rate'],
                                                bins=[-1, thresholds.iloc[0], thresholds.iloc[1], 1],
                                                labels=['difficult', 'medium', 'easy'])

    # # Gán mức độ khó cho các bài
    difficulty_mapping = problem_completion_rate.set_index('problem_id')['difficulty']

    return difficulty_mapping

In [26]:
def get_problems_solve(df: pd.DataFrame)-> dict:
    """Thống kê tỉ lệ bài tập sinh viên giải từng độ khó

    Args:
        df (pd.DataFrame): Assigment DataFrame

    Returns:
        dict: _description_
    """

    # Tính toán số lượng bài theo độ khó cho từng sinh viên
    student_problem_counts = df_cp.groupby(['username', 'difficulty'])['problem_id'].nunique().unstack(fill_value=0)

    # Tính tổng số bài cho từng sinh viên
    student_problem_counts['total'] = student_problem_counts.sum(axis=1)

    # Tính tỉ lệ cho từng mức độ khó (dễ, trung bình, khó)
    student_problem_counts['difficult_rate'] = student_problem_counts['difficult'] / student_problem_counts['total']
    student_problem_counts['medium_rate'] = student_problem_counts['medium'] / student_problem_counts['total']
    student_problem_counts['easy_rate'] = student_problem_counts['easy'] / student_problem_counts['total']

    return student_problem_counts

def get_problems_finished(df: pd.DataFrame)-> dict:
    """Thống kê tỉ lệ problem sinh viên hoàn thành 

    Args:
        df (pd.DataFrame): _description_

    Returns:
        dict: _description_
    """
    student_problem_counts = get_problems_solve(df)

    # Tính toán số lượng bài khó đã giải được cho từng học sinh (có pre_score = 10000)
    student_problem_solved_counts = df_cp[df_cp['pre_score'] == 10000].groupby(['username', 'difficulty'])['problem_id'].nunique().unstack(fill_value=0)

    # Tính tổng số bài khó đã giải được cho từng học sinh
    student_problem_solved_counts['total_solved'] = student_problem_solved_counts.sum(axis=1)

    # Tính tỉ lệ hoàn thành bài khó, trung bình và dễ
    student_problem_solved_counts['difficult_finished_rate'] = student_problem_solved_counts['difficult'] / student_problem_counts.loc[:, 'difficult']
    student_problem_solved_counts['medium_finished_rate'] = student_problem_solved_counts['medium'] / student_problem_counts.loc[:, 'medium']
    student_problem_solved_counts['easy_finished_rate'] = student_problem_solved_counts['easy'] / student_problem_counts.loc[:, 'easy']

    return student_problem_solved_counts

### 3.6 Thống kê số submission nộp trung bình mỗi assginment

In [27]:
def get_avg_submission_per_assignments(df: pd.DataFrame)-> pd.DataFrame:
    """_summary_

    Args:
        df (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """
    # Tính tổng số lần nộp bài của mỗi sinh viên
    total_submissions = df.groupby('username')['problem_id'].count()

    # Tính tổng số assignment của mỗi sinh viên
    total_assignments = df.groupby('username')['assignment_id'].nunique()

    # Chia tổng số lần nộp bài cho tổng số assignment để tính trung bình
    average_submissions_per_assignment = total_submissions / total_assignments

    # Tạo DataFrame để lưu kết quả
    df_average_submissions = average_submissions_per_assignment.reset_index()
    df_average_submissions.columns = ['username', 'avg_submissions_per_assignment']

    # Điền các giá trị NaN bằng 0 (nếu có)
    df_average_submissions['avg_submissions_per_assignment'].fillna(0, inplace=True)

    return df_average_submissions

## 3.7. Thống kê cột created_at

In [28]:
def get_avg_solved_submissions_per_week(df: pd.DataFrame)-> pd.DataFrame:
    """_summary_

    Args:
        df (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """
    # Lọc các bài tập đã được giải (pre_score = 10000)
    solved_problems = df[df['pre_score'] == 10000]

    # Tính tổng số bài tập được giải cho mỗi sinh viên theo tuần
    weekly_solved_submissions = solved_problems.groupby(['username', 'week_number'])['problem_id'].count().reset_index(name='weekly_solved_count')

    # Tính trung bình số bài tập được giải mỗi tuần của mỗi sinh viên
    average_weekly_solved_submissions = weekly_solved_submissions.groupby('username')['weekly_solved_count'].mean().reset_index(name='average_solved_submissions_per_week')

    return average_weekly_solved_submissions
    

def get_avg_submission_per_week(df: pd.DataFrame)-> pd.DataFrame:
    """_summary_

    Args:
        df (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """

    # Tính tổng số bài tập được giải cho mỗi sinh viên theo tuần
    weekly_submissions = df.groupby(['username', 'week_number'])['problem_id'].count().reset_index(name='weekly_count')

    # Tính trung bình số bài tập được giải mỗi tuần của mỗi sinh viên
    average_weekly_solved_submissions = weekly_submissions.groupby('username')['weekly_count'].mean().reset_index(name='avg_submissions_per_week')

    return average_weekly_solved_submissions


## 3.8. Thống kê số submission khó trung bình

In [29]:
def get_submission_difficult(df: pd.DataFrame)-> pd.DataFrame:
    """_summary_

    Args:
        df (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """
    
    average_submissions_difficult = df[df['difficulty'] == 'difficult'].groupby('username').size() / df[df['difficulty'] == 'difficult'].groupby('username')['problem_id'].nunique()
    average_submissions_difficult = average_submissions_difficult.reset_index(name='avg_submissions_difficult')
    average_submissions_difficult.fillna(0, inplace=True)

    return average_submissions_difficult

## 3.9. Thêm đặc trưng vào DataFrame dựa vào DataFrame cũ

In [30]:
# Tạo DataFrame df_statistics để thiết kế đặc trưng
df_statistics = pd.DataFrame()

#Thêm cột tổng số problems & tổng số assignments
df_statistics[['total_problems', 'total_assignments']] = pd.DataFrame(get_total_problems_assignments(df_cp))

#Thêm cột tỉ lệ problem làm đúng sớm, đúng trễ, chưa hoàn thành
df_statistics[['rate_early_correct', 'not_completed']] = pd.DataFrame(get_rate_submission_correct(df_cp))

#Thêm cột tỉ lệ nộp bài thành công, thất bại
df_statistics[['sucessful_rate_avg', 'unsucessful_rate_avg']] = pd.DataFrame(get_successful_unsuccessful_submissions_rate(df_cp))

#Thêm các cột số lượng lỗi và trung bình lỗi trong assignments
err_info_dict = get_error_info(df_cp)
columns = list(err_info_dict.keys())
df_statistics[columns] = pd.DataFrame(err_info_dict)

#Thêm cột phân loại problem khó dễ trung bình vào df_cp
difficulty_mapping = get_difficulty_mapping(df_cp, 0.65, 0.3)
df_cp['difficulty'] = df_cp['problem_id'].map(difficulty_mapping)

#Thêm cột thống kê tỉ lệ problem đã giải dựa vào độ khó
student_problem_counts = get_problems_solve(df_cp)
student_problem_finished = get_problems_finished(df_cp)
df_statistics = df_statistics.merge(student_problem_counts[['difficult_rate', 'easy_rate', 'medium_rate']], left_index=True, right_index=True, how='left')
df_statistics = df_statistics.merge(student_problem_finished[['difficult_finished_rate', 'easy_finished_rate', 'medium_finished_rate']], left_index=True, right_index=True, how='left')
df_statistics = df_statistics.fillna(0)

#Thêm cột tỉ lệ nộp bài trên assignments
avg_submision_per_assignments = get_avg_submission_per_assignments(df_cp)
df_statistics = df_statistics.merge(avg_submision_per_assignments, on='username', how='left')

#Thêm trung bình bài nộp và giải được mỗi tuần 
avg_solved_submissions_per_week = get_avg_solved_submissions_per_week(df_cp)
avg_submissions_per_week = get_avg_submission_per_week(df_cp)
df_statistics = df_statistics.merge(avg_solved_submissions_per_week, on='username', how='left')
df_statistics = df_statistics.merge(avg_submissions_per_week, on='username', how='left')
df_statistics = df_statistics.fillna(0)

#Thêm cột số submission khó trung bình nộp
avg_submissions_difficult = get_submission_difficult(df_cp)
df_statistics = df_statistics.merge(avg_submissions_difficult, on='username', how='left')
df_statistics = df_statistics.fillna(0)

print(df_statistics.isna().sum())
df_statistics.sample(10)

C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\3857062219.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  student_problem_counts = df_cp.groupby(['username', 'difficulty'])['problem_id'].nunique().unstack(fill_value=0)
C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\3857062219.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  student_problem_counts = df_cp.groupby(['username', 'difficulty'])['problem_id'].nunique().unstack(fill_value=0)
C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\3857062219.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a fu

username                               0
total_problems                         0
total_assignments                      0
rate_early_correct                     0
not_completed                          0
sucessful_rate_avg                     0
unsucessful_rate_avg                   0
Limit_Exceeded                         0
Limit_Exceeded_avg                     0
Run_Time_Error                         0
Run_Time_Error_avg                     0
Wrong                                  0
Wrong_avg                              0
Banned                                 0
Banned_avg                             0
difficult_rate                         0
easy_rate                              0
medium_rate                            0
difficult_finished_rate                0
easy_finished_rate                     0
medium_finished_rate                   0
avg_submissions_per_assignment         0
average_solved_submissions_per_week    0
avg_submissions_per_week               0
avg_submissions_

,username,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,...,difficult_rate,easy_rate,medium_rate,difficult_finished_rate,easy_finished_rate,medium_finished_rate,avg_submissions_per_assignment,average_solved_submissions_per_week,avg_submissions_per_week,avg_submissions_difficult
841,8b158f3040b574b6d08148e24946fba8bbdc29ca,60,4,0.900000,0.100000,0.189189,0.810811,13,3.250000,43,...,0.050000,0.516667,0.433333,0.333333,1.000000,0.846154,101.750000,8.555556,45.222222,9.000000
286,2f465acb00ca28cf6a7ef4b71b4df1208eb8fc8c,67,12,0.880597,0.119403,0.468468,0.531532,16,1.333333,14,...,0.074627,0.432836,0.492537,0.400000,0.965517,0.878788,18.500000,9.454545,18.500000,4.000000
225,249c629226d7716b59d1841c45dbca54afe3c1a2,62,8,0.967742,0.032258,0.645833,0.354167,0,0.000000,6,...,0.000000,0.435484,0.564516,0.000000,0.925926,1.000000,12.000000,15.500000,24.000000,0.000000
575,5eebebef2194643799de3dbdbb098cffa856d37d,59,5,0.881356,0.118644,0.146479,0.853521,15,3.000000,41,...,0.016949,0.525424,0.457627,1.000000,0.967742,0.777778,71.000000,5.200000,32.272727,1.000000
1360,e85a95e2eddbdafe7db54351ce3680933e09d0a3,96,10,1.000000,0.000000,0.532609,0.467391,0,0.000000,23,...,0.010417,0.541667,0.447917,1.000000,1.000000,1.000000,18.400000,10.888889,18.400000,1.000000
10,020ef1c31cb4751bbfa1a88fba782db3c402e2fa,62,7,0.935484,0.064516,0.474286,0.525714,11,1.571429,12,...,0.112903,0.500000,0.387097,0.857143,0.935484,0.958333,25.000000,7.545455,15.909091,4.285714
397,4178750f206a91c5b1cf0c5ac6a4021d23912736,38,5,0.894737,0.105263,0.278689,0.721311,4,0.800000,15,...,0.052632,0.473684,0.473684,0.500000,0.888889,0.944444,24.400000,4.857143,15.250000,1.500000
852,8d0a196c1c472a4601384f4235f293c4a341b3a9,56,5,0.982143,0.017857,0.491379,0.508621,8,1.600000,5,...,0.017857,0.553571,0.428571,1.000000,1.000000,0.958333,23.200000,8.142857,16.571429,4.000000
216,232cce96362898f08e9150ba244adaf2d6583ab2,98,11,0.979592,0.020408,0.473684,0.526316,7,0.636364,19,...,0.020408,0.530612,0.448980,0.000000,1.000000,1.000000,20.727273,8.307692,17.538462,3.500000
769,7f8b72d92d190113dbc807befe9e8033ecdda75f,111,12,0.900901,0.099099,0.215953,0.784047,29,2.416667,43,...,0.099099,0.468468,0.432432,0.272727,1.000000,0.937500,42.833333,9.250000,36.714286,9.545455


## Thêm đặc trưng trung bình số lượng submission theo độ khó bài tập

In [31]:
average_submissions_easy = df_cp[df_cp['difficulty'] == 'easy'].groupby('username').size() / df_cp[df_cp['difficulty'] == 'easy'].groupby('username')['problem_id'].nunique()
average_submissions_easy = average_submissions_easy.reset_index(name='avg_submissions_easy')
average_submissions_easy.fillna(0, inplace=True)
df_statistics = df_statistics.merge(average_submissions_easy, on='username', how='left')
df_statistics = df_statistics.fillna(0)


In [32]:
average_submissions_medium = df_cp[df_cp['difficulty'] == 'medium'].groupby('username').size() / df_cp[df_cp['difficulty'] == 'medium'].groupby('username')['problem_id'].nunique()
average_submissions_medium = average_submissions_medium.reset_index(name='average_submissions_medium')
average_submissions_medium.fillna(0, inplace=True)
df_statistics = df_statistics.merge(average_submissions_medium, on='username', how='left')
df_statistics = df_statistics.fillna(0)

## Thêm đặc trưng số lượng submission theo từng tuần trong cột created_at

In [33]:
df_cp['week_number'].nunique()
data = df_cp.pivot(columns='week_number', values='pre_score')
data = data.where(data.isna(), 1)
data.fillna(0, inplace=True)
data = data.rename(columns={col: f'week {col}' for col in data.columns})
data.sum()

week_number
week 1     16994.0
week 2     28112.0
week 3      4102.0
week 4      5787.0
week 5      1973.0
week 6        24.0
week 7        34.0
week 8        16.0
week 9         7.0
week 10        3.0
week 11        8.0
week 12       31.0
week 13       51.0
week 14      877.0
week 15     1429.0
week 16      441.0
week 17      998.0
week 18      169.0
week 19     1562.0
week 20     2133.0
week 21     1200.0
week 36     1275.0
week 37     1061.0
week 38     8768.0
week 39    16828.0
week 40    29519.0
week 41    25052.0
week 42    23296.0
week 43    22219.0
week 44     6767.0
week 45    22859.0
week 46    23983.0
week 47    26003.0
week 48     4981.0
week 49      904.0
week 50      658.0
week 51     1682.0
week 52    13392.0
dtype: float64

In [34]:
week_columns = data.columns.tolist()
df_cp[week_columns] = data

In [35]:
week_sum = df_cp.groupby('username')[week_columns].sum()
df_statistics = df_statistics.merge(week_sum, on='username', how='left')

In [36]:
def assignment_problem_encode(df_stastic, df, assignment_column, problem_column):
    map_to_assignment = df.groupby('username')[assignment_column].mean()
    assignment_encode = df_stastic['username'].map(map_to_assignment)

    map_to_problem = df.groupby('username')[problem_column].mean()
    problem_encode = df_stastic['username'].map(map_to_problem)
    return pd.DataFrame({'assignment_encode': assignment_encode,
                         'problem_encode': problem_encode})

df_statistics[['assignment_encode_tbtl', 'problem_encode_tbtl']] = assignment_problem_encode(df_statistics, df_cp, 'assignment_encode_tbtl', 'problem_encode_tbtl')
df_statistics[['assignment_encode_th', 'problem_encode_th']] = assignment_problem_encode(df_statistics, df_cp, 'assignment_encode_th', 'problem_encode_th')
df_statistics[['assignment_encode_qt', 'problem_encode_qt']] = assignment_problem_encode(df_statistics, df_cp, 'assignment_encode_qt', 'problem_encode_qt')
df_statistics[['assignment_encode_ck', 'problem_encode_ck']] = assignment_problem_encode(df_statistics, df_cp, 'assignment_encode_ck', 'problem_encode_ck')

In [37]:
map_username_total_score = df_cp.groupby('username')['max_pre_score'].sum()
df_statistics['total_score'] = df_statistics['username'].map(map_username_total_score)

In [38]:
map_mean_pre_score_easy = df_cp[df_cp['difficulty'] == 'easy'].groupby('username')['pre_score'].mean()
df_statistics['avg_pre_score_easy'] = df_statistics['username'].map(map_mean_pre_score_easy)

map_mean_pre_score_medium = df_cp[df_cp['difficulty'] == 'medium'].groupby('username')['pre_score'].mean()
df_statistics['avg_pre_score_medium'] = df_statistics['username'].map(map_mean_pre_score_medium)

map_mean_pre_score_difficult = df_cp[df_cp['difficulty'] == 'difficult'].groupby('username')['pre_score'].mean()
df_statistics['avg_pre_score_difficult'] = df_statistics['username'].map(map_mean_pre_score_difficult)

In [39]:
map_user_to_n_problem_first_solve_easy = df_cp[df_cp['difficulty'] == 'easy'].groupby(['username', 'problem_id'])['pre_score'].apply(lambda group: group.size == 1)
map_user_to_n_problem_first_solve_easy = map_user_to_n_problem_first_solve_easy.reset_index(name='num_solve_one_time')
map_user_to_n_problem_first_solve_easy = map_user_to_n_problem_first_solve_easy.groupby('username')['num_solve_one_time'].sum()

df_statistics['count_problem_solved_first_time_easy'] = df_statistics['username'].map(map_user_to_n_problem_first_solve_easy)

map_user_to_n_problem_first_solve_medium = df_cp[df_cp['difficulty'] == 'medium'].groupby(['username', 'problem_id'])['pre_score'].apply(lambda group: group.size == 1)
map_user_to_n_problem_first_solve_medium = map_user_to_n_problem_first_solve_medium.reset_index(name='num_solve_one_time')
map_user_to_n_problem_first_solve_medium = map_user_to_n_problem_first_solve_medium.groupby('username')['num_solve_one_time'].sum()

df_statistics['count_problem_solved_first_time_medium'] = df_statistics['username'].map(map_user_to_n_problem_first_solve_medium)

map_user_to_n_problem_first_solve_difficult = df_cp[df_cp['difficulty'] == 'difficult'].groupby(['username', 'problem_id'])['pre_score'].apply(lambda group: group.size == 1)
map_user_to_n_problem_first_solve_difficult = map_user_to_n_problem_first_solve_difficult.reset_index(name='num_solve_one_time')
map_user_to_n_problem_first_solve_difficult = map_user_to_n_problem_first_solve_difficult.groupby('username')['num_solve_one_time'].sum()

df_statistics['count_problem_solved_first_time_difficult'] = df_statistics['username'].map(map_user_to_n_problem_first_solve_difficult)

In [40]:
data = df_cp.groupby(['username', 'problem_id'])['pre_score'].apply(lambda group: group.size == 1)
data = data.reset_index(name='is_one_time_to_solve')
df_cp = df_cp.merge(data, on=['username', 'problem_id'], how='left')

In [41]:
def map_user_to_max_avg_improve_score(difficult):
    def max_improve_score(group):
        list_score = list(group)
        list_improve_score = [list_score[i + 1] - list_score[i] for i in range(len(list_score) - 1)]
        return max(list_improve_score)

    def mean_improve_score(group):
        list_score = list(group)
        list_improve_score = [list_score[i + 1] - list_score[i] for i in range(len(list_score) - 1)]
        return sum(list_improve_score) / len(list_improve_score)

    avg_max_improve_score = df_cp[(df_cp['is_one_time_to_solve'] == False) & (df_cp['difficulty'] == difficult)].groupby(['username', 'problem_id'])['pre_score'].apply(max_improve_score)
    avg_max_improve_score = avg_max_improve_score.reset_index(name='max_improve_score')
    map_user_to_avg_max_improve_score = avg_max_improve_score.groupby('username')['max_improve_score'].mean()

    avg_improve_score_easy = df_cp[(df_cp['is_one_time_to_solve'] == False) & (df_cp['difficulty'] == difficult)].groupby(['username', 'problem_id'])['pre_score'].apply(mean_improve_score)
    avg_improve_score_easy = avg_improve_score_easy.reset_index(name='avg_improve_score')
    map_user_to_avg_improve_score = avg_improve_score_easy.groupby('username')['avg_improve_score'].mean()

    return {'max': map_user_to_avg_max_improve_score,
            'avg': map_user_to_avg_improve_score}
map_max_easy = map_user_to_max_avg_improve_score('easy')['max']
map_avg_easy = map_user_to_max_avg_improve_score('easy')['avg']
df_statistics['avg_max_improve_score_easy'] = df_statistics['username'].map(map_max_easy)
df_statistics['avg_improve_score_easy'] = df_statistics['username'].map(map_avg_easy)

map_max_medium = map_user_to_max_avg_improve_score('medium')['max']
map_avg_medium = map_user_to_max_avg_improve_score('medium')['avg']
df_statistics['avg_max_improve_score_medium'] = df_statistics['username'].map(map_max_medium)
df_statistics['avg_improve_score_medium'] = df_statistics['username'].map(map_avg_medium)

map_max_difficult = map_user_to_max_avg_improve_score('difficult')['max']
map_avg_difficult = map_user_to_max_avg_improve_score('difficult')['avg']
df_statistics['avg_max_improve_score_difficult'] = df_statistics['username'].map(map_max_difficult)
df_statistics['avg_improve_score_difficult'] = df_statistics['username'].map(map_avg_difficult)

#### Thiết kế thêm features dựa theo created_at và updated_at

In [42]:
df_statistics.columns

Index(['username', 'total_problems', 'total_assignments', 'rate_early_correct',
       'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg',
       'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error',
       'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg',
       'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate',
       'easy_finished_rate', 'medium_finished_rate',
       'avg_submissions_per_assignment', 'average_solved_submissions_per_week',
       'avg_submissions_per_week', 'avg_submissions_difficult',
       'avg_submissions_easy', 'average_submissions_medium', 'week 1',
       'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8',
       'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14',
       'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20',
       'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40',
       'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week

# 4. Lựa chọn các feature phù hợp và xem xét độ tương quan giữa các feature

## 4.1. QT

In [43]:
# Tạo bản sao của df_statistics để phân tích
full_df_temp_qt = df_statistics.copy()

In [44]:
columns_drop_qt = ['assignment_encode_tbtl', 'problem_encode_tbtl', 
                   'assignment_encode_th', 'problem_encode_th', 
                    'assignment_encode_ck', 'problem_encode_ck']

full_df_temp_qt = full_df_temp_qt.drop(columns=columns_drop_qt)

In [45]:
full_df_temp_qt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 80 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   username                                   1489 non-null   object 
 1   total_problems                             1489 non-null   int64  
 2   total_assignments                          1489 non-null   int64  
 3   rate_early_correct                         1489 non-null   float64
 4   not_completed                              1489 non-null   float64
 5   sucessful_rate_avg                         1489 non-null   float64
 6   unsucessful_rate_avg                       1489 non-null   float64
 7   Limit_Exceeded                             1489 non-null   int64  
 8   Limit_Exceeded_avg                         1489 non-null   float64
 9   Run_Time_Error                             1489 non-null   int64  
 10  Run_Time_Error_avg      

In [46]:
full_df_temp_qt

,username,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,...,avg_pre_score_difficult,count_problem_solved_first_time_easy,count_problem_solved_first_time_medium,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,46,7,1.000000,0.000000,0.367347,0.632653,6,0.857143,16,...,7693.266667,12.0,4.0,1.0,6714.285714,3380.952381,7644.166667,2909.730159,6149.75,3392.875000
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,78,9,0.923077,0.076923,0.328185,0.671815,5,0.555556,23,...,5216.181818,18.0,8.0,1.0,6391.571429,3344.539683,5925.230769,3092.358364,7089.00,1796.732143
2,01122b3ef7e59b84189e65985305f575d6bdf83c,66,7,0.878788,0.121212,0.302564,0.697436,18,2.571429,12,...,4779.766667,20.0,16.0,5.0,8387.857143,3775.667749,6705.153846,3868.391026,3495.40,533.618681
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,47,4,1.000000,0.000000,0.520000,0.480000,1,0.250000,14,...,6500.000000,18.0,9.0,0.0,7357.142857,4390.476190,6522.400000,3061.200000,6000.00,0.000000
4,013de369c439ab0ead8aa7da64423aa395a8be39,52,8,0.846154,0.153846,0.551402,0.448598,6,0.750000,9,...,2495.666667,14.0,10.0,4.0,3846.153846,1211.538462,6088.000000,3750.952381,1003.75,169.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,34,9,1.000000,0.000000,0.540541,0.459459,0,0.000000,3,...,2857.142857,12.0,7.0,1.0,8125.000000,2031.250000,8400.000000,6400.000000,10000.00,2000.000000
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,77,6,0.961039,0.038961,0.307692,0.692308,1,0.166667,14,...,3888.761905,29.0,8.0,0.0,8050.000000,4461.111111,6379.107143,2471.707711,7000.00,755.555556
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,100,6,0.990000,0.010000,0.240275,0.759725,22,3.666667,18,...,2669.352941,26.0,13.0,1.0,7882.964286,4094.995310,7023.466667,2352.358590,6730.50,958.875000
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,34,7,0.882353,0.117647,0.319149,0.680851,7,1.000000,9,...,3069.833333,7.0,6.0,1.0,9382.777778,7901.296296,7604.111111,2343.186949,526.00,263.000000


## 4.2. TH

In [47]:
# Tạo bản sao của df_statistics để phân tích
full_df_temp_th = df_statistics.copy()

In [48]:
columns_drop_th = ['assignment_encode_tbtl', 'problem_encode_tbtl', 
                   'assignment_encode_qt', 'problem_encode_qt',
                    'assignment_encode_ck', 'problem_encode_ck']

full_df_temp_th = full_df_temp_th.drop(columns=columns_drop_th)

In [49]:
full_df_temp_th

,username,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,...,avg_pre_score_difficult,count_problem_solved_first_time_easy,count_problem_solved_first_time_medium,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,46,7,1.000000,0.000000,0.367347,0.632653,6,0.857143,16,...,7693.266667,12.0,4.0,1.0,6714.285714,3380.952381,7644.166667,2909.730159,6149.75,3392.875000
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,78,9,0.923077,0.076923,0.328185,0.671815,5,0.555556,23,...,5216.181818,18.0,8.0,1.0,6391.571429,3344.539683,5925.230769,3092.358364,7089.00,1796.732143
2,01122b3ef7e59b84189e65985305f575d6bdf83c,66,7,0.878788,0.121212,0.302564,0.697436,18,2.571429,12,...,4779.766667,20.0,16.0,5.0,8387.857143,3775.667749,6705.153846,3868.391026,3495.40,533.618681
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,47,4,1.000000,0.000000,0.520000,0.480000,1,0.250000,14,...,6500.000000,18.0,9.0,0.0,7357.142857,4390.476190,6522.400000,3061.200000,6000.00,0.000000
4,013de369c439ab0ead8aa7da64423aa395a8be39,52,8,0.846154,0.153846,0.551402,0.448598,6,0.750000,9,...,2495.666667,14.0,10.0,4.0,3846.153846,1211.538462,6088.000000,3750.952381,1003.75,169.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,34,9,1.000000,0.000000,0.540541,0.459459,0,0.000000,3,...,2857.142857,12.0,7.0,1.0,8125.000000,2031.250000,8400.000000,6400.000000,10000.00,2000.000000
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,77,6,0.961039,0.038961,0.307692,0.692308,1,0.166667,14,...,3888.761905,29.0,8.0,0.0,8050.000000,4461.111111,6379.107143,2471.707711,7000.00,755.555556
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,100,6,0.990000,0.010000,0.240275,0.759725,22,3.666667,18,...,2669.352941,26.0,13.0,1.0,7882.964286,4094.995310,7023.466667,2352.358590,6730.50,958.875000
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,34,7,0.882353,0.117647,0.319149,0.680851,7,1.000000,9,...,3069.833333,7.0,6.0,1.0,9382.777778,7901.296296,7604.111111,2343.186949,526.00,263.000000


## 4.3. TBTL

In [50]:
# Tạo bản sao của df_statistics để phân tích
full_df_temp_tbtl = df_statistics.copy()

In [51]:
columns_drop_tbtl = ['assignment_encode_th', 'problem_encode_th', 
                   'assignment_encode_qt', 'problem_encode_qt',
                    'assignment_encode_ck', 'problem_encode_ck']

full_df_temp_tbtl = full_df_temp_tbtl.drop(columns=columns_drop_tbtl)

In [52]:
columns_drop = ['not_completed',
 'week 11',
 'week 12',
 'week 8',
 'week 9',
 'week 10',
 'week 3',
 'week 4',
 'week 14',
 'week 6',
 'week 7',
 'week 50',
 'week 51',
 'week 49']

In [53]:
full_df_temp_tbtl = full_df_temp_tbtl.drop(columns=columns_drop)


In [54]:
full_df_temp_tbtl

,username,total_problems,total_assignments,rate_early_correct,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,...,avg_pre_score_difficult,count_problem_solved_first_time_easy,count_problem_solved_first_time_medium,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,46,7,1.000000,0.367347,0.632653,6,0.857143,16,2.285714,...,7693.266667,12.0,4.0,1.0,6714.285714,3380.952381,7644.166667,2909.730159,6149.75,3392.875000
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,78,9,0.923077,0.328185,0.671815,5,0.555556,23,2.555556,...,5216.181818,18.0,8.0,1.0,6391.571429,3344.539683,5925.230769,3092.358364,7089.00,1796.732143
2,01122b3ef7e59b84189e65985305f575d6bdf83c,66,7,0.878788,0.302564,0.697436,18,2.571429,12,1.714286,...,4779.766667,20.0,16.0,5.0,8387.857143,3775.667749,6705.153846,3868.391026,3495.40,533.618681
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,47,4,1.000000,0.520000,0.480000,1,0.250000,14,3.500000,...,6500.000000,18.0,9.0,0.0,7357.142857,4390.476190,6522.400000,3061.200000,6000.00,0.000000
4,013de369c439ab0ead8aa7da64423aa395a8be39,52,8,0.846154,0.551402,0.448598,6,0.750000,9,1.125000,...,2495.666667,14.0,10.0,4.0,3846.153846,1211.538462,6088.000000,3750.952381,1003.75,169.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,34,9,1.000000,0.540541,0.459459,0,0.000000,3,0.333333,...,2857.142857,12.0,7.0,1.0,8125.000000,2031.250000,8400.000000,6400.000000,10000.00,2000.000000
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,77,6,0.961039,0.307692,0.692308,1,0.166667,14,2.333333,...,3888.761905,29.0,8.0,0.0,8050.000000,4461.111111,6379.107143,2471.707711,7000.00,755.555556
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,100,6,0.990000,0.240275,0.759725,22,3.666667,18,3.000000,...,2669.352941,26.0,13.0,1.0,7882.964286,4094.995310,7023.466667,2352.358590,6730.50,958.875000
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,34,7,0.882353,0.319149,0.680851,7,1.000000,9,1.285714,...,3069.833333,7.0,6.0,1.0,9382.777778,7901.296296,7604.111111,2343.186949,526.00,263.000000


## 4.4. CK

In [55]:
# Tạo bản sao của df_statistics để phân tích
full_df_temp_ck = df_statistics.copy()

In [56]:
columns_drop_ck = ['assignment_encode_tbtl', 'problem_encode_tbtl', 
                   'assignment_encode_th', 'problem_encode_th', 
                   'assignment_encode_qt', 'problem_encode_qt']

full_df_temp_ck = full_df_temp_ck.drop(columns=columns_drop_ck)

In [57]:
full_df_temp_ck

,username,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,...,avg_pre_score_difficult,count_problem_solved_first_time_easy,count_problem_solved_first_time_medium,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,46,7,1.000000,0.000000,0.367347,0.632653,6,0.857143,16,...,7693.266667,12.0,4.0,1.0,6714.285714,3380.952381,7644.166667,2909.730159,6149.75,3392.875000
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,78,9,0.923077,0.076923,0.328185,0.671815,5,0.555556,23,...,5216.181818,18.0,8.0,1.0,6391.571429,3344.539683,5925.230769,3092.358364,7089.00,1796.732143
2,01122b3ef7e59b84189e65985305f575d6bdf83c,66,7,0.878788,0.121212,0.302564,0.697436,18,2.571429,12,...,4779.766667,20.0,16.0,5.0,8387.857143,3775.667749,6705.153846,3868.391026,3495.40,533.618681
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,47,4,1.000000,0.000000,0.520000,0.480000,1,0.250000,14,...,6500.000000,18.0,9.0,0.0,7357.142857,4390.476190,6522.400000,3061.200000,6000.00,0.000000
4,013de369c439ab0ead8aa7da64423aa395a8be39,52,8,0.846154,0.153846,0.551402,0.448598,6,0.750000,9,...,2495.666667,14.0,10.0,4.0,3846.153846,1211.538462,6088.000000,3750.952381,1003.75,169.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,34,9,1.000000,0.000000,0.540541,0.459459,0,0.000000,3,...,2857.142857,12.0,7.0,1.0,8125.000000,2031.250000,8400.000000,6400.000000,10000.00,2000.000000
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,77,6,0.961039,0.038961,0.307692,0.692308,1,0.166667,14,...,3888.761905,29.0,8.0,0.0,8050.000000,4461.111111,6379.107143,2471.707711,7000.00,755.555556
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,100,6,0.990000,0.010000,0.240275,0.759725,22,3.666667,18,...,2669.352941,26.0,13.0,1.0,7882.964286,4094.995310,7023.466667,2352.358590,6730.50,958.875000
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,34,7,0.882353,0.117647,0.319149,0.680851,7,1.000000,9,...,3069.833333,7.0,6.0,1.0,9382.777778,7901.296296,7604.111111,2343.186949,526.00,263.000000


# 5. Xử lý missing values và scale data

## 5.1. QT

In [58]:
# Lựa chọn các cột kiểu dữ liệu phân loại (categorical) từ DataFrame, ngoại trừ cột "username"
categorical_columns_df_qt = full_df_temp_qt.select_dtypes(include=['object']).drop(["username"], axis=1).columns

# Lựa chọn các cột kiểu dữ liệu số (numerical) từ DataFrame
numerical_columns_df_qt = full_df_temp_qt.select_dtypes(np.number).columns

In [59]:
categorical_columns_df_qt

Index([], dtype='object')

In [60]:
numerical_columns_df_qt

Index(['total_problems', 'total_assignments', 'rate_early_correct',
       'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg',
       'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error',
       'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg',
       'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate',
       'easy_finished_rate', 'medium_finished_rate',
       'avg_submissions_per_assignment', 'average_solved_submissions_per_week',
       'avg_submissions_per_week', 'avg_submissions_difficult',
       'avg_submissions_easy', 'average_submissions_medium', 'week 1',
       'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8',
       'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14',
       'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20',
       'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40',
       'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46',
      

In [61]:
columns_qt = categorical_columns_df_qt.tolist() + numerical_columns_df_qt.tolist() + ['username']
columns_qt

['total_problems',
 'total_assignments',
 'rate_early_correct',
 'not_completed',
 'sucessful_rate_avg',
 'unsucessful_rate_avg',
 'Limit_Exceeded',
 'Limit_Exceeded_avg',
 'Run_Time_Error',
 'Run_Time_Error_avg',
 'Wrong',
 'Wrong_avg',
 'Banned',
 'Banned_avg',
 'difficult_rate',
 'easy_rate',
 'medium_rate',
 'difficult_finished_rate',
 'easy_finished_rate',
 'medium_finished_rate',
 'avg_submissions_per_assignment',
 'average_solved_submissions_per_week',
 'avg_submissions_per_week',
 'avg_submissions_difficult',
 'avg_submissions_easy',
 'average_submissions_medium',
 'week 1',
 'week 2',
 'week 3',
 'week 4',
 'week 5',
 'week 6',
 'week 7',
 'week 8',
 'week 9',
 'week 10',
 'week 11',
 'week 12',
 'week 13',
 'week 14',
 'week 15',
 'week 16',
 'week 17',
 'week 18',
 'week 19',
 'week 20',
 'week 21',
 'week 36',
 'week 37',
 'week 38',
 'week 39',
 'week 40',
 'week 41',
 'week 42',
 'week 43',
 'week 44',
 'week 45',
 'week 46',
 'week 47',
 'week 48',
 'week 49',
 'week 50'

In [62]:
# Định nghĩa bộ xử lý cho các cột số
numerical_transformer_qt = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),  # Điền giá trị missing bằng 0
    ('scaler', StandardScaler())  # Chuẩn hóa các giá trị số
])

In [63]:
# Định nghĩa bộ xử lý cho các cột phân loại
categorical_transformer_qt = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Điền giá trị missing bằng 'missing'
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encode các giá trị phân loại
])

In [64]:
# Kết hợp các bộ xử lý số và phân loại vào một bộ xử lý cột tổng hợp
preprocessor_df_qt = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer_qt, numerical_columns_df_qt),  # Áp dụng bộ xử lý số
        ('cat', categorical_transformer_qt, categorical_columns_df_qt)  # Áp dụng bộ xử lý phân loại
    ],
    remainder='passthrough'  # Giữ nguyên các cột không được chỉ định trong num hoặc cat transformers
)

In [65]:
# Tạo Pipeline kết hợp các bước tiền xử lý
pipeline_df_qt = Pipeline(steps=[
    ('preprocessor', preprocessor_df_qt)  # Áp dụng bộ tiền xử lý
])

In [66]:
# Áp dụng pipeline tiền xử lý lên DataFrame
X_qt = full_df_temp_qt
X_preprocessed_qt = pipeline_df_qt.fit_transform(X_qt)

In [67]:

df_temp_1_qt = pd.DataFrame(X_preprocessed_qt, columns=columns_qt)


## 5.2. TH

In [68]:
# Lựa chọn các cột kiểu dữ liệu phân loại (categorical) từ DataFrame, ngoại trừ cột "username"
categorical_columns_df_th = full_df_temp_th.select_dtypes(include=['object']).drop(["username"], axis=1).columns

# Lựa chọn các cột kiểu dữ liệu số (numerical) từ DataFrame
numerical_columns_df_th = full_df_temp_th.select_dtypes(np.number).columns

In [69]:
categorical_columns_df_th

Index([], dtype='object')

In [70]:
numerical_columns_df_th

Index(['total_problems', 'total_assignments', 'rate_early_correct',
       'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg',
       'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error',
       'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg',
       'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate',
       'easy_finished_rate', 'medium_finished_rate',
       'avg_submissions_per_assignment', 'average_solved_submissions_per_week',
       'avg_submissions_per_week', 'avg_submissions_difficult',
       'avg_submissions_easy', 'average_submissions_medium', 'week 1',
       'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8',
       'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14',
       'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20',
       'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40',
       'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46',
      

In [71]:
columns_th = categorical_columns_df_th.tolist() + numerical_columns_df_th.tolist() + ['username']
columns_th

['total_problems',
 'total_assignments',
 'rate_early_correct',
 'not_completed',
 'sucessful_rate_avg',
 'unsucessful_rate_avg',
 'Limit_Exceeded',
 'Limit_Exceeded_avg',
 'Run_Time_Error',
 'Run_Time_Error_avg',
 'Wrong',
 'Wrong_avg',
 'Banned',
 'Banned_avg',
 'difficult_rate',
 'easy_rate',
 'medium_rate',
 'difficult_finished_rate',
 'easy_finished_rate',
 'medium_finished_rate',
 'avg_submissions_per_assignment',
 'average_solved_submissions_per_week',
 'avg_submissions_per_week',
 'avg_submissions_difficult',
 'avg_submissions_easy',
 'average_submissions_medium',
 'week 1',
 'week 2',
 'week 3',
 'week 4',
 'week 5',
 'week 6',
 'week 7',
 'week 8',
 'week 9',
 'week 10',
 'week 11',
 'week 12',
 'week 13',
 'week 14',
 'week 15',
 'week 16',
 'week 17',
 'week 18',
 'week 19',
 'week 20',
 'week 21',
 'week 36',
 'week 37',
 'week 38',
 'week 39',
 'week 40',
 'week 41',
 'week 42',
 'week 43',
 'week 44',
 'week 45',
 'week 46',
 'week 47',
 'week 48',
 'week 49',
 'week 50'

In [72]:
# Định nghĩa bộ xử lý cho các cột số
numerical_transformer_th = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),  # Điền giá trị missing bằng 0
    ('scaler', StandardScaler())  # Chuẩn hóa các giá trị số
])

In [73]:
# Định nghĩa bộ xử lý cho các cột phân loại
categorical_transformer_th = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Điền giá trị missing bằng 'missing'
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encode các giá trị phân loại
])

In [74]:
# Kết hợp các bộ xử lý số và phân loại vào một bộ xử lý cột tổng hợp
preprocessor_df_th = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer_th, numerical_columns_df_th),  # Áp dụng bộ xử lý số
        ('cat', categorical_transformer_th, categorical_columns_df_th)  # Áp dụng bộ xử lý phân loại
    ],
    remainder='passthrough'  # Giữ nguyên các cột không được chỉ định trong num hoặc cat transformers
)

In [75]:
# Tạo Pipeline kết hợp các bước tiền xử lý
pipeline_df_th = Pipeline(steps=[
    ('preprocessor', preprocessor_df_th)  # Áp dụng bộ tiền xử lý
])

In [76]:
# Áp dụng pipeline tiền xử lý lên DataFrame
X_th = full_df_temp_th
X_preprocessed_th = pipeline_df_th.fit_transform(X_th)

In [77]:
# Chuyển đổi dữ liệu đã tiền xử lý thành DataFrame và đặt tên các cột
df_temp_1_th = pd.DataFrame(X_preprocessed_th, columns=columns_th)

## 5.3. TBTL

In [78]:
# Lựa chọn các cột kiểu dữ liệu phân loại (categorical) từ DataFrame, ngoại trừ cột "username"
categorical_columns_df_tbtl = full_df_temp_tbtl.select_dtypes(include=['object']).drop(["username"], axis=1).columns

# Lựa chọn các cột kiểu dữ liệu số (numerical) từ DataFrame
numerical_columns_df_tbtl = full_df_temp_tbtl.select_dtypes(np.number).columns

In [79]:
categorical_columns_df_tbtl

Index([], dtype='object')

In [80]:
numerical_columns_df_tbtl

Index(['total_problems', 'total_assignments', 'rate_early_correct',
       'sucessful_rate_avg', 'unsucessful_rate_avg', 'Limit_Exceeded',
       'Limit_Exceeded_avg', 'Run_Time_Error', 'Run_Time_Error_avg', 'Wrong',
       'Wrong_avg', 'Banned', 'Banned_avg', 'difficult_rate', 'easy_rate',
       'medium_rate', 'difficult_finished_rate', 'easy_finished_rate',
       'medium_finished_rate', 'avg_submissions_per_assignment',
       'average_solved_submissions_per_week', 'avg_submissions_per_week',
       'avg_submissions_difficult', 'avg_submissions_easy',
       'average_submissions_medium', 'week 1', 'week 2', 'week 5', 'week 13',
       'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20',
       'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40',
       'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46',
       'week 47', 'week 48', 'week 52', 'assignment_encode_tbtl',
       'problem_encode_tbtl', 'total_score', 'avg_pre_score_easy',
       '

In [81]:
columns_tbtl = categorical_columns_df_tbtl.tolist() + numerical_columns_df_tbtl.tolist() + ['username']
columns_tbtl

['total_problems',
 'total_assignments',
 'rate_early_correct',
 'sucessful_rate_avg',
 'unsucessful_rate_avg',
 'Limit_Exceeded',
 'Limit_Exceeded_avg',
 'Run_Time_Error',
 'Run_Time_Error_avg',
 'Wrong',
 'Wrong_avg',
 'Banned',
 'Banned_avg',
 'difficult_rate',
 'easy_rate',
 'medium_rate',
 'difficult_finished_rate',
 'easy_finished_rate',
 'medium_finished_rate',
 'avg_submissions_per_assignment',
 'average_solved_submissions_per_week',
 'avg_submissions_per_week',
 'avg_submissions_difficult',
 'avg_submissions_easy',
 'average_submissions_medium',
 'week 1',
 'week 2',
 'week 5',
 'week 13',
 'week 15',
 'week 16',
 'week 17',
 'week 18',
 'week 19',
 'week 20',
 'week 21',
 'week 36',
 'week 37',
 'week 38',
 'week 39',
 'week 40',
 'week 41',
 'week 42',
 'week 43',
 'week 44',
 'week 45',
 'week 46',
 'week 47',
 'week 48',
 'week 52',
 'assignment_encode_tbtl',
 'problem_encode_tbtl',
 'total_score',
 'avg_pre_score_easy',
 'avg_pre_score_medium',
 'avg_pre_score_difficult',

In [82]:
# Định nghĩa bộ xử lý cho các cột số
numerical_transformer_tbtl = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),  # Điền giá trị missing bằng 0
    ('scaler', StandardScaler())  # Chuẩn hóa các giá trị số
])

In [83]:
# Định nghĩa bộ xử lý cho các cột phân loại
categorical_transformer_tbtl = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Điền giá trị missing bằng 'missing'
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encode các giá trị phân loại
])

In [84]:
# Kết hợp các bộ xử lý số và phân loại vào một bộ xử lý cột tổng hợp
preprocessor_df_tbtl = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer_tbtl, numerical_columns_df_tbtl),  # Áp dụng bộ xử lý số
        ('cat', categorical_transformer_tbtl, categorical_columns_df_tbtl)  # Áp dụng bộ xử lý phân loại
    ],
    remainder='passthrough'  # Giữ nguyên các cột không được chỉ định trong num hoặc cat transformers
)

In [85]:
# Tạo Pipeline kết hợp các bước tiền xử lý
pipeline_df_tbtl = Pipeline(steps=[
    ('preprocessor', preprocessor_df_tbtl)  # Áp dụng bộ tiền xử lý
])

In [86]:
# Áp dụng pipeline tiền xử lý lên DataFrame
X_tbtl = full_df_temp_tbtl
X_preprocessed_tbtl = pipeline_df_tbtl.fit_transform(X_tbtl)

In [87]:
# Chuyển đổi dữ liệu đã tiền xử lý thành DataFrame và đặt tên các cột
df_temp_1_tbtl = pd.DataFrame(X_preprocessed_tbtl, columns = columns_tbtl)

## 5.4. CK

In [88]:
# Lựa chọn các cột kiểu dữ liệu phân loại (categorical) từ DataFrame, ngoại trừ cột "username"
categorical_columns_df_ck = full_df_temp_ck.select_dtypes(include=['object']).drop(["username"], axis=1).columns

# Lựa chọn các cột kiểu dữ liệu số (numerical) từ DataFrame
numerical_columns_df_ck = full_df_temp_ck.select_dtypes(np.number).columns

In [89]:
categorical_columns_df_ck

Index([], dtype='object')

In [90]:
numerical_columns_df_ck

Index(['total_problems', 'total_assignments', 'rate_early_correct',
       'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg',
       'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error',
       'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg',
       'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate',
       'easy_finished_rate', 'medium_finished_rate',
       'avg_submissions_per_assignment', 'average_solved_submissions_per_week',
       'avg_submissions_per_week', 'avg_submissions_difficult',
       'avg_submissions_easy', 'average_submissions_medium', 'week 1',
       'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8',
       'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14',
       'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20',
       'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40',
       'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46',
      

In [91]:
columns_ck = categorical_columns_df_ck.tolist() + numerical_columns_df_ck.tolist() + ['username']
columns_ck

['total_problems',
 'total_assignments',
 'rate_early_correct',
 'not_completed',
 'sucessful_rate_avg',
 'unsucessful_rate_avg',
 'Limit_Exceeded',
 'Limit_Exceeded_avg',
 'Run_Time_Error',
 'Run_Time_Error_avg',
 'Wrong',
 'Wrong_avg',
 'Banned',
 'Banned_avg',
 'difficult_rate',
 'easy_rate',
 'medium_rate',
 'difficult_finished_rate',
 'easy_finished_rate',
 'medium_finished_rate',
 'avg_submissions_per_assignment',
 'average_solved_submissions_per_week',
 'avg_submissions_per_week',
 'avg_submissions_difficult',
 'avg_submissions_easy',
 'average_submissions_medium',
 'week 1',
 'week 2',
 'week 3',
 'week 4',
 'week 5',
 'week 6',
 'week 7',
 'week 8',
 'week 9',
 'week 10',
 'week 11',
 'week 12',
 'week 13',
 'week 14',
 'week 15',
 'week 16',
 'week 17',
 'week 18',
 'week 19',
 'week 20',
 'week 21',
 'week 36',
 'week 37',
 'week 38',
 'week 39',
 'week 40',
 'week 41',
 'week 42',
 'week 43',
 'week 44',
 'week 45',
 'week 46',
 'week 47',
 'week 48',
 'week 49',
 'week 50'

In [92]:
# Định nghĩa bộ xử lý cho các cột số
numerical_transformer_ck = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),  # Điền giá trị missing bằng 0
    ('scaler', StandardScaler())  # Chuẩn hóa các giá trị số
])

In [93]:
# Định nghĩa bộ xử lý cho các cột phân loại
categorical_transformer_ck = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Điền giá trị missing bằng 'missing'
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encode các giá trị phân loại
])

In [94]:
# Kết hợp các bộ xử lý số và phân loại vào một bộ xử lý cột tổng hợp
preprocessor_df_ck = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer_ck, numerical_columns_df_ck),  # Áp dụng bộ xử lý số
        ('cat', categorical_transformer_ck, categorical_columns_df_ck)  # Áp dụng bộ xử lý phân loại
    ],
    remainder='passthrough'  # Giữ nguyên các cột không được chỉ định trong num hoặc cat transformers
)

In [95]:
# Tạo Pipeline kết hợp các bước tiền xử lý
pipeline_df_ck = Pipeline(steps=[
    ('preprocessor', preprocessor_df_ck)  # Áp dụng bộ tiền xử lý
])

In [96]:
# Áp dụng pipeline tiền xử lý lên DataFrame
X_ck = full_df_temp_ck
X_preprocessed_ck = pipeline_df_ck.fit_transform(X_ck)

In [97]:
# Chuyển đổi dữ liệu đã tiền xử lý thành DataFrame và đặt tên các cột
df_temp_1_ck = pd.DataFrame(X_preprocessed_ck, columns=columns_ck)

# 6. Chuẩn bị dữ liệu cho việc huấn luyện mô hình và dự đoán

## 6.1. QT

In [98]:
full_df_qt = pd.merge(df_temp_1_qt, df_qt, left_on='username', right_on='hash', how='left')

In [99]:
public_df_qt = full_df_qt[full_df_qt['username'].isin(df_qt['hash'])]

In [100]:
# Chuyển đổi cột 'diemqt' sang kiểu số, các giá trị không phải số sẽ thành NaN
public_df_qt['diemqt'] = pd.to_numeric(public_df_qt['diemqt'], errors='coerce')

# Điền các giá trị NaN bằng 0 (hoặc giá trị mặc định khác nếu bạn muốn)
public_df_qt.fillna(0, inplace=True)

C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\591799820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_df_qt['diemqt'] = pd.to_numeric(public_df_qt['diemqt'], errors='coerce')
C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\591799820.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_df_qt.fillna(0, inplace=True)


In [101]:
public_df_qt

,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,...,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,hash,diemqt
0,-0.501164,-0.061584,0.672419,-0.672419,0.168114,0.052935,-0.337106,-0.322165,-0.049084,-0.07914,...,-0.085698,-0.121415,-0.336302,0.628901,0.012822,0.634171,1.226412,00b6dd4fc7eb817e03708c532016ef30ce564a61,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.5
1,0.464209,0.468462,0.2818,-0.2818,-0.108019,0.281601,-0.402789,-0.469686,0.372335,0.017735,...,-0.085698,-0.256571,-0.356706,-0.151804,0.135385,0.922597,0.343747,00bef8afee8f3c595d535c9c03c490cac1a4f021,00bef8afee8f3c595d535c9c03c490cac1a4f021,7.0
2,0.102194,-0.061584,0.056898,-0.056898,-0.288676,0.431205,0.451092,0.516376,-0.289895,-0.284287,...,2.505247,0.579489,-0.115117,0.202421,0.656185,-0.180931,-0.354754,01122b3ef7e59b84189e65985305f575d6bdf83c,01122b3ef7e59b84189e65985305f575d6bdf83c,9.0
4,-0.320156,0.203439,-0.108818,0.108818,1.465903,-1.021768,-0.337106,-0.374574,-0.470503,-0.495845,...,1.857511,-1.32261,-1.551967,-0.077878,0.577371,-0.94607,-0.556008,013de369c439ab0ead8aa7da64423aa395a8be39,013de369c439ab0ead8aa7da64423aa395a8be39,10.0
5,0.826223,0.468462,0.051769,-0.051769,0.377094,-0.120122,-0.731205,-0.741435,0.191727,-0.101934,...,3.80072,0.90566,0.640642,-0.324573,-0.54929,-1.254303,-0.649845,014c59c6433fd764a0b08de6ffeb757eaf60aa73,014c59c6433fd764a0b08de6ffeb757eaf60aa73,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,-0.591667,-0.326607,0.200043,-0.200043,0.182374,0.041126,-0.599839,-0.578386,-0.049084,0.057625,...,-0.085698,0.341457,-0.419787,0.637909,0.207629,-0.333059,-0.373345,840d07858c03f80f4695056e2cc7d0c474b83a25,840d07858c03f80f4695056e2cc7d0c474b83a25,9.0
797,1.248574,2.588646,0.574764,-0.574764,-0.28898,0.431456,-0.271423,-0.540021,1.335579,-0.076123,...,1.209775,0.396687,-0.321122,1.115238,1.060404,0.918861,1.610355,844f5db2e7e31ae51eba025480679ed7e4708ac6,844f5db2e7e31ae51eba025480679ed7e4708ac6,8.0
798,-0.320156,0.203439,0.184145,-0.184145,-0.284011,0.427341,-0.599839,-0.619148,0.071321,-0.091962,...,-0.085698,0.273884,-0.606269,1.335529,0.610257,-1.254303,-0.649845,845acd04a77b3d1b623f255d9f9f8eae90892dab,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.5
799,1.550253,1.263531,0.449697,-0.449697,-0.991612,1.013307,0.779508,0.1961,2.720241,0.955144,...,-0.733434,0.413177,0.15267,0.728014,0.067128,-0.139188,0.298364,8460eaaf887a6289fb156f7562fb739ba8e9629e,8460eaaf887a6289fb156f7562fb739ba8e9629e,10.0


In [102]:
private_df_qt = full_df_qt[~full_df_qt['username'].isin(df_qt['hash'])]

In [103]:
private_df_qt

,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,...,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,hash,diemqt
3,-0.470996,-0.856652,0.672419,-0.672419,1.244485,-0.838411,-0.665522,-0.619148,-0.169489,0.356798,...,-0.733434,0.147818,0.229401,0.119418,0.114474,0.588186,-0.649845,0134f9f410c65ad0e8c2254a7e9288670e02a183,NaN,NaN
20,-0.380492,-0.856652,0.672419,-0.672419,0.124144,0.089347,-0.731205,-0.741435,0.131524,0.805557,...,-0.085698,1.044028,1.052675,-0.295255,-0.580448,-1.254303,-0.649845,035f97702f2c01d26ab1fae8f39ea2f98a0caa3c,NaN,NaN
68,-0.380492,-0.856652,0.672419,-0.672419,-0.475689,0.58607,-0.271423,0.114575,0.071321,0.715805,...,-0.733434,0.986039,0.869257,0.979246,-0.636048,0.281104,-0.017846,0aaebc88f6106684d6993c156104c1ef36cf94e0,NaN,NaN
80,0.072026,-0.326607,0.438047,-0.438047,0.377094,-0.120122,-0.074373,0.073813,-0.289895,-0.181713,...,-0.085698,-0.412111,-0.343407,0.011117,0.243349,0.402939,0.391963,0bf111a9caedf02804f6991792490e63bc21058a,NaN,NaN
120,0.041858,-0.59163,0.593074,-0.593074,0.71679,-0.401425,-0.599839,-0.545776,-0.4103,-0.181713,...,-0.733434,-0.387059,0.074633,-0.093926,-1.092109,-0.947222,-0.373345,12887fd9a4df4ba9b88a71f3fb1d2502a75995dd,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,-0.863178,0.468462,0.672419,-0.672419,1.389319,-0.958348,-0.731205,-0.741435,-0.831719,-0.780059,...,-0.085698,0.469402,-1.092629,0.972185,2.355162,1.816512,0.456153,feb8a2859a011c59efd22ed419cb69288fe03627,NaN,NaN
1485,0.434041,-0.326607,0.474573,-0.474573,-0.252517,0.401261,-0.665522,-0.659911,-0.169489,-0.062044,...,-0.733434,0.437992,0.268982,0.054337,-0.281137,0.895267,-0.232023,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,NaN,NaN
1486,1.127902,-0.326607,0.621638,-0.621638,-0.727886,0.794915,0.713825,1.05211,0.071321,0.177294,...,-0.085698,0.368036,0.063823,0.346992,-0.361233,0.812509,-0.119588,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,NaN,NaN
1487,-0.863178,-0.061584,0.075002,-0.075002,-0.171735,0.334365,-0.271423,-0.252287,-0.470503,-0.438147,...,-0.085698,0.996169,2.196744,0.610709,-0.367388,-1.092779,-0.504406,ff3fa2ec64294f37ae968159f810ebeda7966c51,NaN,NaN


In [104]:
public_df_qt = public_df_qt[public_df_qt['diemqt'] >= 5.5]

In [105]:
from sklearn.model_selection import train_test_split

X_train_qt = public_df_qt.iloc[:, 0:-3].values
y_train_qt = public_df_qt.iloc[:, -1].values

## 6.2. TH

In [106]:
full_df_th = pd.merge(df_temp_1_th, df_th, left_on='username', right_on='hash', how='left')

In [107]:
public_df_th = full_df_th[full_df_th['username'].isin(df_th['hash'])]


In [108]:
# Chuyển đổi cột 'TH' sang kiểu số, các giá trị không phải số sẽ thành NaN
public_df_th['TH'] = pd.to_numeric(public_df_th['TH'], errors='coerce')

# Điền các giá trị NaN bằng 0 (hoặc giá trị mặc định khác nếu bạn muốn)
public_df_th.fillna(0, inplace=True)

C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\1385570457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_df_th['TH'] = pd.to_numeric(public_df_th['TH'], errors='coerce')
C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\1385570457.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_df_th.fillna(0, inplace=True)


In [109]:
private_df_th = full_df_th[~full_df_th['username'].isin(df_th['hash'])]
private_df_th

,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,...,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,hash,TH
3,-0.470996,-0.856652,0.672419,-0.672419,1.244485,-0.838411,-0.665522,-0.619148,-0.169489,0.356798,...,-0.733434,0.147818,0.229401,0.119418,0.114474,0.588186,-0.649845,0134f9f410c65ad0e8c2254a7e9288670e02a183,NaN,NaN
20,-0.380492,-0.856652,0.672419,-0.672419,0.124144,0.089347,-0.731205,-0.741435,0.131524,0.805557,...,-0.085698,1.044028,1.052675,-0.295255,-0.580448,-1.254303,-0.649845,035f97702f2c01d26ab1fae8f39ea2f98a0caa3c,NaN,NaN
68,-0.380492,-0.856652,0.672419,-0.672419,-0.475689,0.58607,-0.271423,0.114575,0.071321,0.715805,...,-0.733434,0.986039,0.869257,0.979246,-0.636048,0.281104,-0.017846,0aaebc88f6106684d6993c156104c1ef36cf94e0,NaN,NaN
80,0.072026,-0.326607,0.438047,-0.438047,0.377094,-0.120122,-0.074373,0.073813,-0.289895,-0.181713,...,-0.085698,-0.412111,-0.343407,0.011117,0.243349,0.402939,0.391963,0bf111a9caedf02804f6991792490e63bc21058a,NaN,NaN
120,0.041858,-0.59163,0.593074,-0.593074,0.71679,-0.401425,-0.599839,-0.545776,-0.4103,-0.181713,...,-0.733434,-0.387059,0.074633,-0.093926,-1.092109,-0.947222,-0.373345,12887fd9a4df4ba9b88a71f3fb1d2502a75995dd,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,-0.863178,0.468462,0.672419,-0.672419,1.389319,-0.958348,-0.731205,-0.741435,-0.831719,-0.780059,...,-0.085698,0.469402,-1.092629,0.972185,2.355162,1.816512,0.456153,feb8a2859a011c59efd22ed419cb69288fe03627,NaN,NaN
1485,0.434041,-0.326607,0.474573,-0.474573,-0.252517,0.401261,-0.665522,-0.659911,-0.169489,-0.062044,...,-0.733434,0.437992,0.268982,0.054337,-0.281137,0.895267,-0.232023,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,NaN,NaN
1486,1.127902,-0.326607,0.621638,-0.621638,-0.727886,0.794915,0.713825,1.05211,0.071321,0.177294,...,-0.085698,0.368036,0.063823,0.346992,-0.361233,0.812509,-0.119588,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,NaN,NaN
1487,-0.863178,-0.061584,0.075002,-0.075002,-0.171735,0.334365,-0.271423,-0.252287,-0.470503,-0.438147,...,-0.085698,0.996169,2.196744,0.610709,-0.367388,-1.092779,-0.504406,ff3fa2ec64294f37ae968159f810ebeda7966c51,NaN,NaN


In [110]:
from sklearn.model_selection import train_test_split

X_train_th = public_df_th.iloc[:, 0:-3].values
y_train_th = public_df_th.iloc[:, -1].values

## 6.3. TBTL

In [111]:
full_df_tbtl = pd.merge(df_temp_1_tbtl, df_tbtl, on='username', how='left')

In [112]:
public_df_tbtl = full_df_tbtl[full_df_tbtl['username'].isin(df_tbtl['username'])]


In [113]:
public_df_tbtl = public_df_tbtl[public_df_tbtl['TBTL'] >= 5]

In [114]:
public_df_tbtl

,total_problems,total_assignments,rate_early_correct,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,Wrong,...,count_problem_solved_first_time_medium,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,TBTL
0,-0.501164,-0.061584,0.672419,0.168114,0.052935,-0.337106,-0.322165,-0.049084,-0.07914,-0.342364,...,-0.839786,-0.085698,-0.121415,-0.336302,0.628901,0.012822,0.634171,1.226412,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.24
1,0.464209,0.468462,0.2818,-0.108019,0.281601,-0.402789,-0.469686,0.372335,0.017735,0.163872,...,-0.308982,-0.085698,-0.256571,-0.356706,-0.151804,0.135385,0.922597,0.343747,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.11
2,0.102194,-0.061584,0.056898,-0.288676,0.431205,0.451092,0.516376,-0.289895,-0.284287,0.357433,...,0.752626,2.505247,0.579489,-0.115117,0.202421,0.656185,-0.180931,-0.354754,01122b3ef7e59b84189e65985305f575d6bdf83c,7.30
3,-0.470996,-0.856652,0.672419,1.244485,-0.838411,-0.665522,-0.619148,-0.169489,0.356798,-0.893267,...,-0.176281,-0.733434,0.147818,0.229401,0.119418,0.114474,0.588186,-0.649845,0134f9f410c65ad0e8c2254a7e9288670e02a183,8.63
4,-0.320156,0.203439,-0.108818,1.465903,-1.021768,-0.337106,-0.374574,-0.470503,-0.495845,-0.848599,...,-0.04358,1.857511,-1.32261,-1.551967,-0.077878,0.577371,-0.94607,-0.556008,013de369c439ab0ead8aa7da64423aa395a8be39,8.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,-0.591667,-0.326607,0.200043,0.182374,0.041126,-0.599839,-0.578386,-0.049084,0.057625,-0.684817,...,-0.441683,-0.085698,0.341457,-0.419787,0.637909,0.207629,-0.333059,-0.373345,840d07858c03f80f4695056e2cc7d0c474b83a25,7.67
797,1.248574,2.588646,0.574764,-0.28898,0.431456,-0.271423,-0.540021,1.335579,-0.076123,0.774333,...,-0.176281,1.209775,0.396687,-0.321122,1.115238,1.060404,0.918861,1.610355,844f5db2e7e31ae51eba025480679ed7e4708ac6,7.54
798,-0.320156,0.203439,0.184145,-0.284011,0.427341,-0.599839,-0.619148,0.071321,-0.091962,-0.253028,...,0.487224,-0.085698,0.273884,-0.606269,1.335529,0.610257,-1.254303,-0.649845,845acd04a77b3d1b623f255d9f9f8eae90892dab,7.49
799,1.550253,1.263531,0.449697,-0.991612,1.013307,0.779508,0.1961,2.720241,0.955144,2.814165,...,0.221822,-0.733434,0.413177,0.15267,0.728014,0.067128,-0.139188,0.298364,8460eaaf887a6289fb156f7562fb739ba8e9629e,8.94


In [115]:
private_df_tbtl = full_df_tbtl[~full_df_tbtl['username'].isin(df_tbtl['username'])]
private_df_tbtl

,total_problems,total_assignments,rate_early_correct,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,Wrong,...,count_problem_solved_first_time_medium,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,TBTL
394,-1.4062,-1.386698,0.672419,0.365558,-0.110569,-0.731205,-0.741435,-0.891922,-0.540721,-1.086828,...,-0.972487,-0.733434,0.207648,-0.829957,1.536617,0.936277,-1.254303,-0.649845,410357eb9129023509cfaf8d38be61c050bb3b05,NaN
625,-1.828551,-1.651721,-4.405622,-2.422086,-3.641147,-0.731205,-0.741435,-0.711314,0.895309,-1.191053,...,-1.37059,-0.085698,-2.933408,-2.230871,-2.842923,-1.939915,-1.254303,-0.649845,67212308d026508fd5b6942ffbbdd7b0be2e89de,NaN
801,1.067566,-0.326607,0.672419,1.839567,-1.3312,-0.731205,-0.741435,-0.470503,-0.361217,-0.506146,...,1.814233,-0.085698,0.355462,0.852513,-0.525472,-0.984719,1.816512,4.880147,84a17972cc6d29489bbe205a9e7feb8745726fbc,NaN
802,0.58488,-0.061584,0.362782,1.015325,-0.648642,-0.337106,-0.322165,-0.109287,-0.130427,-0.580592,...,0.885327,1.209775,0.259999,0.690299,0.357713,0.906612,-0.073268,-0.543393,84b6b2d70924066c8345f2bc2281791ae3188da2,NaN
803,0.977063,1.263531,0.405153,-0.663843,0.741881,-0.140056,-0.374574,1.275376,0.237129,1.101897,...,0.089121,0.562039,0.371236,0.044511,0.252258,-0.466264,0.59013,-0.292638,851d9a4b9b8e236f2d62282ddf06fae57b7d9492,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,-0.863178,0.468462,0.672419,1.389319,-0.958348,-0.731205,-0.741435,-0.831719,-0.780059,-0.774153,...,-0.441683,-0.085698,0.469402,-1.092629,0.972185,2.355162,1.816512,0.456153,feb8a2859a011c59efd22ed419cb69288fe03627,NaN
1485,0.434041,-0.326607,0.474573,-0.252517,0.401261,-0.665522,-0.659911,-0.169489,-0.062044,0.714776,...,-0.308982,-0.733434,0.437992,0.268982,0.054337,-0.281137,0.895267,-0.232023,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,NaN
1486,1.127902,-0.326607,0.621638,-0.727886,0.794915,0.713825,1.05211,0.071321,0.177294,2.173926,...,0.354523,-0.085698,0.368036,0.063823,0.346992,-0.361233,0.812509,-0.119588,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,NaN
1487,-0.863178,-0.061584,0.075002,-0.171735,0.334365,-0.271423,-0.252287,-0.470503,-0.438147,-0.655039,...,-0.574384,-0.085698,0.996169,2.196744,0.610709,-0.367388,-1.092779,-0.504406,ff3fa2ec64294f37ae968159f810ebeda7966c51,NaN


In [116]:
X_train_tbtl = public_df_tbtl.iloc[:, 0:-2].values
y_train_tbtl = public_df_tbtl.iloc[:, -1].values

In [117]:
public_df_tbtl.columns

Index(['total_problems', 'total_assignments', 'rate_early_correct',
       'sucessful_rate_avg', 'unsucessful_rate_avg', 'Limit_Exceeded',
       'Limit_Exceeded_avg', 'Run_Time_Error', 'Run_Time_Error_avg', 'Wrong',
       'Wrong_avg', 'Banned', 'Banned_avg', 'difficult_rate', 'easy_rate',
       'medium_rate', 'difficult_finished_rate', 'easy_finished_rate',
       'medium_finished_rate', 'avg_submissions_per_assignment',
       'average_solved_submissions_per_week', 'avg_submissions_per_week',
       'avg_submissions_difficult', 'avg_submissions_easy',
       'average_submissions_medium', 'week 1', 'week 2', 'week 5', 'week 13',
       'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20',
       'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40',
       'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46',
       'week 47', 'week 48', 'week 52', 'assignment_encode_tbtl',
       'problem_encode_tbtl', 'total_score', 'avg_pre_score_easy',
       '

## 6.4. CK

In [118]:
full_df_ck = pd.merge(df_temp_1_ck, df_ck, left_on='username', right_on='hash', how='left')

In [119]:
public_df_ck = full_df_ck[full_df_ck['username'].isin(df_ck['hash'])]
public_df_ck

,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,...,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,hash,CK
0,-0.501164,-0.061584,0.672419,-0.672419,0.168114,0.052935,-0.337106,-0.322165,-0.049084,-0.07914,...,-0.085698,-0.121415,-0.336302,0.628901,0.012822,0.634171,1.226412,00b6dd4fc7eb817e03708c532016ef30ce564a61,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.5
1,0.464209,0.468462,0.2818,-0.2818,-0.108019,0.281601,-0.402789,-0.469686,0.372335,0.017735,...,-0.085698,-0.256571,-0.356706,-0.151804,0.135385,0.922597,0.343747,00bef8afee8f3c595d535c9c03c490cac1a4f021,00bef8afee8f3c595d535c9c03c490cac1a4f021,6.5
2,0.102194,-0.061584,0.056898,-0.056898,-0.288676,0.431205,0.451092,0.516376,-0.289895,-0.284287,...,2.505247,0.579489,-0.115117,0.202421,0.656185,-0.180931,-0.354754,01122b3ef7e59b84189e65985305f575d6bdf83c,01122b3ef7e59b84189e65985305f575d6bdf83c,4.5
4,-0.320156,0.203439,-0.108818,0.108818,1.465903,-1.021768,-0.337106,-0.374574,-0.470503,-0.495845,...,1.857511,-1.32261,-1.551967,-0.077878,0.577371,-0.94607,-0.556008,013de369c439ab0ead8aa7da64423aa395a8be39,013de369c439ab0ead8aa7da64423aa395a8be39,5.0
5,0.826223,0.468462,0.051769,-0.051769,0.377094,-0.120122,-0.731205,-0.741435,0.191727,-0.101934,...,3.80072,0.90566,0.640642,-0.324573,-0.54929,-1.254303,-0.649845,014c59c6433fd764a0b08de6ffeb757eaf60aa73,014c59c6433fd764a0b08de6ffeb757eaf60aa73,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,-0.591667,-0.326607,0.200043,-0.200043,0.182374,0.041126,-0.599839,-0.578386,-0.049084,0.057625,...,-0.085698,0.341457,-0.419787,0.637909,0.207629,-0.333059,-0.373345,840d07858c03f80f4695056e2cc7d0c474b83a25,840d07858c03f80f4695056e2cc7d0c474b83a25,5.5
797,1.248574,2.588646,0.574764,-0.574764,-0.28898,0.431456,-0.271423,-0.540021,1.335579,-0.076123,...,1.209775,0.396687,-0.321122,1.115238,1.060404,0.918861,1.610355,844f5db2e7e31ae51eba025480679ed7e4708ac6,844f5db2e7e31ae51eba025480679ed7e4708ac6,6.0
798,-0.320156,0.203439,0.184145,-0.184145,-0.284011,0.427341,-0.599839,-0.619148,0.071321,-0.091962,...,-0.085698,0.273884,-0.606269,1.335529,0.610257,-1.254303,-0.649845,845acd04a77b3d1b623f255d9f9f8eae90892dab,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.0
799,1.550253,1.263531,0.449697,-0.449697,-0.991612,1.013307,0.779508,0.1961,2.720241,0.955144,...,-0.733434,0.413177,0.15267,0.728014,0.067128,-0.139188,0.298364,8460eaaf887a6289fb156f7562fb739ba8e9629e,8460eaaf887a6289fb156f7562fb739ba8e9629e,7.0


In [120]:
# Chuyển đổi cột 'CK' sang kiểu số, các giá trị không phải số sẽ thành NaN
public_df_ck['CK'] = pd.to_numeric(public_df_ck['CK'], errors='coerce')

# Điền các giá trị NaN bằng 0 (hoặc giá trị mặc định khác nếu bạn muốn)
public_df_ck.fillna(0, inplace=True)


C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\1689644307.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_df_ck['CK'] = pd.to_numeric(public_df_ck['CK'], errors='coerce')
C:\Users\hoivd\AppData\Local\Temp\ipykernel_23796\1689644307.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_df_ck.fillna(0, inplace=True)


In [121]:
public_df_ck

,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,...,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,hash,CK
0,-0.501164,-0.061584,0.672419,-0.672419,0.168114,0.052935,-0.337106,-0.322165,-0.049084,-0.07914,...,-0.085698,-0.121415,-0.336302,0.628901,0.012822,0.634171,1.226412,00b6dd4fc7eb817e03708c532016ef30ce564a61,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.5
1,0.464209,0.468462,0.2818,-0.2818,-0.108019,0.281601,-0.402789,-0.469686,0.372335,0.017735,...,-0.085698,-0.256571,-0.356706,-0.151804,0.135385,0.922597,0.343747,00bef8afee8f3c595d535c9c03c490cac1a4f021,00bef8afee8f3c595d535c9c03c490cac1a4f021,6.5
2,0.102194,-0.061584,0.056898,-0.056898,-0.288676,0.431205,0.451092,0.516376,-0.289895,-0.284287,...,2.505247,0.579489,-0.115117,0.202421,0.656185,-0.180931,-0.354754,01122b3ef7e59b84189e65985305f575d6bdf83c,01122b3ef7e59b84189e65985305f575d6bdf83c,4.5
4,-0.320156,0.203439,-0.108818,0.108818,1.465903,-1.021768,-0.337106,-0.374574,-0.470503,-0.495845,...,1.857511,-1.32261,-1.551967,-0.077878,0.577371,-0.94607,-0.556008,013de369c439ab0ead8aa7da64423aa395a8be39,013de369c439ab0ead8aa7da64423aa395a8be39,5.0
5,0.826223,0.468462,0.051769,-0.051769,0.377094,-0.120122,-0.731205,-0.741435,0.191727,-0.101934,...,3.80072,0.90566,0.640642,-0.324573,-0.54929,-1.254303,-0.649845,014c59c6433fd764a0b08de6ffeb757eaf60aa73,014c59c6433fd764a0b08de6ffeb757eaf60aa73,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,-0.591667,-0.326607,0.200043,-0.200043,0.182374,0.041126,-0.599839,-0.578386,-0.049084,0.057625,...,-0.085698,0.341457,-0.419787,0.637909,0.207629,-0.333059,-0.373345,840d07858c03f80f4695056e2cc7d0c474b83a25,840d07858c03f80f4695056e2cc7d0c474b83a25,5.5
797,1.248574,2.588646,0.574764,-0.574764,-0.28898,0.431456,-0.271423,-0.540021,1.335579,-0.076123,...,1.209775,0.396687,-0.321122,1.115238,1.060404,0.918861,1.610355,844f5db2e7e31ae51eba025480679ed7e4708ac6,844f5db2e7e31ae51eba025480679ed7e4708ac6,6.0
798,-0.320156,0.203439,0.184145,-0.184145,-0.284011,0.427341,-0.599839,-0.619148,0.071321,-0.091962,...,-0.085698,0.273884,-0.606269,1.335529,0.610257,-1.254303,-0.649845,845acd04a77b3d1b623f255d9f9f8eae90892dab,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.0
799,1.550253,1.263531,0.449697,-0.449697,-0.991612,1.013307,0.779508,0.1961,2.720241,0.955144,...,-0.733434,0.413177,0.15267,0.728014,0.067128,-0.139188,0.298364,8460eaaf887a6289fb156f7562fb739ba8e9629e,8460eaaf887a6289fb156f7562fb739ba8e9629e,7.0


In [122]:
private_df_ck = full_df_ck[~full_df_ck['username'].isin(df_ck['hash'])]
private_df_ck

,total_problems,total_assignments,rate_early_correct,not_completed,sucessful_rate_avg,unsucessful_rate_avg,Limit_Exceeded,Limit_Exceeded_avg,Run_Time_Error,Run_Time_Error_avg,...,count_problem_solved_first_time_difficult,avg_max_improve_score_easy,avg_improve_score_easy,avg_max_improve_score_medium,avg_improve_score_medium,avg_max_improve_score_difficult,avg_improve_score_difficult,username,hash,CK
3,-0.470996,-0.856652,0.672419,-0.672419,1.244485,-0.838411,-0.665522,-0.619148,-0.169489,0.356798,...,-0.733434,0.147818,0.229401,0.119418,0.114474,0.588186,-0.649845,0134f9f410c65ad0e8c2254a7e9288670e02a183,NaN,NaN
20,-0.380492,-0.856652,0.672419,-0.672419,0.124144,0.089347,-0.731205,-0.741435,0.131524,0.805557,...,-0.085698,1.044028,1.052675,-0.295255,-0.580448,-1.254303,-0.649845,035f97702f2c01d26ab1fae8f39ea2f98a0caa3c,NaN,NaN
68,-0.380492,-0.856652,0.672419,-0.672419,-0.475689,0.58607,-0.271423,0.114575,0.071321,0.715805,...,-0.733434,0.986039,0.869257,0.979246,-0.636048,0.281104,-0.017846,0aaebc88f6106684d6993c156104c1ef36cf94e0,NaN,NaN
80,0.072026,-0.326607,0.438047,-0.438047,0.377094,-0.120122,-0.074373,0.073813,-0.289895,-0.181713,...,-0.085698,-0.412111,-0.343407,0.011117,0.243349,0.402939,0.391963,0bf111a9caedf02804f6991792490e63bc21058a,NaN,NaN
120,0.041858,-0.59163,0.593074,-0.593074,0.71679,-0.401425,-0.599839,-0.545776,-0.4103,-0.181713,...,-0.733434,-0.387059,0.074633,-0.093926,-1.092109,-0.947222,-0.373345,12887fd9a4df4ba9b88a71f3fb1d2502a75995dd,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,-0.863178,0.468462,0.672419,-0.672419,1.389319,-0.958348,-0.731205,-0.741435,-0.831719,-0.780059,...,-0.085698,0.469402,-1.092629,0.972185,2.355162,1.816512,0.456153,feb8a2859a011c59efd22ed419cb69288fe03627,NaN,NaN
1485,0.434041,-0.326607,0.474573,-0.474573,-0.252517,0.401261,-0.665522,-0.659911,-0.169489,-0.062044,...,-0.733434,0.437992,0.268982,0.054337,-0.281137,0.895267,-0.232023,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,NaN,NaN
1486,1.127902,-0.326607,0.621638,-0.621638,-0.727886,0.794915,0.713825,1.05211,0.071321,0.177294,...,-0.085698,0.368036,0.063823,0.346992,-0.361233,0.812509,-0.119588,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,NaN,NaN
1487,-0.863178,-0.061584,0.075002,-0.075002,-0.171735,0.334365,-0.271423,-0.252287,-0.470503,-0.438147,...,-0.085698,0.996169,2.196744,0.610709,-0.367388,-1.092779,-0.504406,ff3fa2ec64294f37ae968159f810ebeda7966c51,NaN,NaN


In [123]:
X_train_ck = public_df_ck.iloc[:, 0:-3].values
y_train_ck = public_df_ck.iloc[:, -1].values

# 7. Build Model


## 7.1 Chọn model để GridSearch

In [124]:
models = {
     'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

In [125]:
# Define the hyperparameter grids for each model
param_grids = {
    'RandomForest': {
        'n_estimators': [400, 500, 700, 800, 900],
        'max_depth': [15, 20, 25],
        'min_samples_split': [5, 10, 20, 40]
    },
    'XGBoost': {
        'n_estimators': [100, 300, 400, 500],
        'learning_rate': [0.001, 0.003, 0.01, 0.03, 0.1],
        'max_depth': [2, 3, 5],
    },
    'GradientBoosting': {
        'n_estimators': [100, 200, 300, 400, 500],
        'learning_rate': [0.001, 0.003, 0.01, 0.03, 0.1],
        'max_depth': [3, 4, 5]
    }
}

### QT

#### Tìm siêu tham số thích hợp nhất cho model bằng cách dùng hàm GridSearchCV của sklearn.

In [126]:
# 3-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define R^2 scorer
r2_scorer = make_scorer(r2_score)

# Train and tune the models
grids_qt = {}
best_models_qt = {}
grid_search_times_qt = {}
for model_name, model in models.items():
    print(f'Training and tuning {model_name}...')

    start_time = time.time()
    grids_qt[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring=r2_scorer, n_jobs=-1, verbose=2)
    grids_qt[model_name].fit(X_train_qt, y_train_qt)
    elapsed_time = time.time() - start_time
    grid_search_times_qt[model_name] = elapsed_time

    best_params_qt = grids_qt[model_name].best_params_
    best_score_qt = grids_qt[model_name].best_score_

    print(f'Best parameters for {model_name}: {best_params_qt}')
    print(f'Best R^2 for {model_name}: {best_score_qt}\n')

    # Save the best model
    best_models_qt[model_name] = grids_qt[model_name].best_estimator_

    # In tổng thời gian chạy
print("\nGrid Search times (in seconds):")
for model_name, elapsed_time in grid_search_times_qt.items():
    print(f'{model_name}: {elapsed_time:.2f} seconds')

Training and tuning RandomForest...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for RandomForest: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 900}
Best R^2 for RandomForest: 0.5186923533378405

Training and tuning XGBoost...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
Best R^2 for XGBoost: 0.5146040645012911

Training and tuning GradientBoosting...
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters for GradientBoosting: {'learning_rate': 0.03, 'max_depth': 4, 'n_estimators': 200}
Best R^2 for GradientBoosting: 0.51572619609318


Grid Search times (in seconds):
RandomForest: 563.03 seconds
XGBoost: 62.96 seconds
GradientBoosting: 309.36 seconds


### TH

#### Tìm siêu tham số thích hợp nhất cho model bằng cách dùng hàm GridSearchCV của sklearn.

In [ ]:
# 3-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define R^2 scorer
r2_scorer = make_scorer(r2_score)

# Train and tune the models
grids_th = {}
best_models_th = {}
grid_search_times_th = {}

for model_name, model in models.items():
    print(f'Training and tuning {model_name}...')

    start_time = time.time()
    grids_th[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring=r2_scorer, n_jobs=-1, verbose=2)
    grids_th[model_name].fit(X_train_th, y_train_th)
    elapsed_time = time.time() - start_time
    grid_search_times_th[model_name] = elapsed_time

    best_params_th = grids_th[model_name].best_params_
    best_score_th = grids_th[model_name].best_score_

    print(f'Best parameters for {model_name}: {best_params_th}')
    print(f'Best R^2 for {model_name}: {best_score_th}\n')

    # Save the best model
    best_models_th[model_name] = grids_th[model_name].best_estimator_
   
# In tổng thời gian chạy
print("\nGrid Search times (in seconds):")
for model_name, elapsed_time in grid_search_times_th.items():
    print(f'{model_name}: {elapsed_time:.2f} seconds')

Training and tuning RandomForest...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for RandomForest: {'max_depth': 15, 'min_samples_split': 5, 'n_estimators': 800}
Best R^2 for RandomForest: 0.5017131539328024

Training and tuning XGBoost...
Fitting 5 folds for each of 60 candidates, totalling 300 fits


### TBTL

#### Tìm siêu tham số thích hợp nhất cho model bằng cách dùng hàm GridSearchCV của sklearn.

In [276]:
# 3-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define R^2 scorer
r2_scorer = make_scorer(r2_score)

# Train and tune the models
grids_tbtl = {}
best_models_tbtl = {}
grid_search_times_tbtl = {}

for model_name, model in models.items():
    print(f'Training and tuning {model_name}...')
    
    start_time = time.time()
    grids_tbtl[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring=r2_scorer, n_jobs=-1, verbose=2)
    grids_tbtl[model_name].fit(X_train_tbtl, y_train_tbtl)
    elapsed_time = time.time() - start_time
    grid_search_times_tbtl[model_name] = elapsed_time

    best_params_tbtl = grids_tbtl[model_name].best_params_
    best_score_tbtl = grids_tbtl[model_name].best_score_

    print(f'Best parameters for {model_name}: {best_params_tbtl}')
    print(f'Best R^2 for {model_name}: {best_score_tbtl}\n')

    # Save the best model
    best_models_tbtl[model_name] = grids_tbtl[model_name].best_estimator_
    
# In tổng thời gian chạy
print("\nGrid Search times (in seconds):")
for model_name, elapsed_time in grid_search_times_tbtl.items():
    print(f'{model_name}: {elapsed_time:.2f} seconds')

Training and tuning RandomForest...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for RandomForest: {'max_depth': 25, 'min_samples_split': 10, 'n_estimators': 500}
Best R^2 for RandomForest: 0.3206005735020302

Training and tuning XGBoost...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 400}
Best R^2 for XGBoost: 0.31881180373727147

Training and tuning GradientBoosting...
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters for GradientBoosting: {'learning_rate': 0.03, 'max_depth': 4, 'n_estimators': 100}
Best R^2 for GradientBoosting: 0.32697964057719103


Grid Search times (in seconds):
RandomForest: 586.09 seconds
XGBoost: 65.95 seconds
GradientBoosting: 274.30 seconds


### CK

#### Tìm siêu tham số thích hợp nhất cho model bằng cách dùng hàm GridSearchCV của sklearn.

In [277]:
# 3-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define R^2 scorer
r2_scorer = make_scorer(r2_score)

# Train and tune the models
grids_ck = {}
best_models_ck = {}
grid_search_times_ck = {}

for model_name, model in models.items():
    print(f'Training and tuning {model_name}...')
    start_time = time.time()
    
    grids_ck[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring=r2_scorer, n_jobs=-1, verbose=2)
    grids_ck[model_name].fit(X_train_ck, y_train_ck)
    elapsed_time = time.time() - start_time
    grid_search_times_ck[model_name] = elapsed_time

    best_params_ck = grids_ck[model_name].best_params_
    best_score_ck = grids_ck[model_name].best_score_

    print(f'Best parameters for {model_name}: {best_params_ck}')
    print(f'Best R^2 for {model_name}: {best_score_ck}\n')

    # Save the best model
    best_models_ck[model_name] = grids_ck[model_name].best_estimator_
    
# In tổng thời gian chạy
print("\nGrid Search times (in seconds):")
for model_name, elapsed_time in grid_search_times_ck.items():
    print(f'{model_name}: {elapsed_time:.2f} seconds')

Training and tuning RandomForest...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for RandomForest: {'max_depth': 25, 'min_samples_split': 5, 'n_estimators': 500}
Best R^2 for RandomForest: 0.4212985514997312

Training and tuning XGBoost...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
Best R^2 for XGBoost: 0.4352926100096828

Training and tuning GradientBoosting...
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters for GradientBoosting: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
Best R^2 for GradientBoosting: 0.4216791126074758


Grid Search times (in seconds):
RandomForest: 704.76 seconds
XGBoost: 69.05 seconds
GradientBoosting: 263.39 seconds


# Prediction

### QT

In [278]:
y_pred_qt = []
X_private_qt = private_df_qt.iloc[:, 0:-3].values
for i in grids_qt.keys():
    y_pred_qt.append(grids_qt[i].predict(X_private_qt))

In [279]:
private_df_qt.columns

Index(['total_problems', 'total_assignments', 'rate_early_correct',
       'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg',
       'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error',
       'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg',
       'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate',
       'easy_finished_rate', 'medium_finished_rate',
       'avg_submissions_per_assignment', 'average_solved_submissions_per_week',
       'avg_submissions_per_week', 'avg_submissions_difficult',
       'avg_submissions_easy', 'average_submissions_medium', 'week 1',
       'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8',
       'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14',
       'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20',
       'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40',
       'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46',
      

In [280]:
grids_qt.keys()

dict_keys(['RandomForest', 'XGBoost', 'GradientBoosting'])

In [281]:
y_pred_qt

[array([8.17214322, 7.8838004 , 7.91999759, 8.6941448 , 8.13254795,
        8.02051045, 7.50191408, 8.2543788 , 8.01880572, 8.08212691,
        8.20660284, 8.27958817, 7.96569699, 7.91893845, 8.76356006,
        7.88913986, 8.23133947, 8.10018326, 7.77893372, 8.01284833,
        7.80729902, 8.16830492, 8.33968764, 7.85996965, 8.01750783,
        7.95419978, 8.18153444, 7.78181504, 7.95591751, 9.56198258,
        8.07424184, 7.92682857, 7.60380305, 8.11347426, 8.00439799,
        8.07375226, 7.97996517, 7.88948888, 7.5938949 , 7.83938437,
        7.24632816, 9.16723371, 7.3636147 , 7.5431343 , 9.3024404 ,
        9.35731702, 7.66834121, 9.70529167, 7.69453806, 8.84940295,
        7.99165823, 9.00002249, 9.15756996, 8.73765032, 6.85792404,
        9.42943078, 8.59443276, 9.66552668, 9.35659165, 8.96768577,
        9.19122321, 9.1110191 , 6.60916578, 7.3419255 , 7.1446812 ,
        8.87510686, 8.79427934, 8.64438865, 7.30693893, 8.99825333,
        7.03611121, 7.26511071, 8.8571794 , 8.19

In [282]:
import os

path = './predict/qt'
os.makedirs(path, exist_ok=True)

for i, key in enumerate(grids_qt.keys()):
  result_qt = pd.merge(private_df_qt['username'].reset_index(drop=True), pd.DataFrame(y_pred_qt[i], columns=['predict']), left_index=True, right_index=True, how='left').reset_index(drop=True)
  file_path = f'./predict/qt/{key}_qt.csv'
  result_qt.to_csv(file_path, header=False, index=False)    

### TH

In [283]:
y_pred_th = []
X_private_th = private_df_th.iloc[:, 0:-3].values
for i in grids_th.keys():
    y_pred_th.append(grids_th[i].predict(X_private_th))

In [284]:
X_private_th.shape

(728, 79)

In [285]:
y_pred_th

[array([6.50686778, 6.43635698, 6.69095981, 8.44977104, 6.2222709 ,
        6.64450846, 4.66638534, 6.76726673, 6.90475788, 6.29338143,
        6.2179373 , 6.91809576, 5.41573842, 3.81764998, 3.48028444,
        6.61907664, 5.37644034, 6.46447429, 6.10624891, 6.38950374,
        5.82110679, 6.52438516, 4.10533084, 6.46785025, 6.68640717,
        6.50166134, 6.59940428, 5.30140439, 4.94152829, 6.15660107,
        6.2757655 , 6.04861495, 5.46373999, 6.56926682, 4.54054469,
        6.74924896, 6.59951674, 6.05816715, 2.78464361, 5.2973696 ,
        5.4970651 , 8.13833994, 8.09339832, 7.12684053, 9.07086655,
        8.70955617, 6.21972722, 6.78518323, 7.20472577, 7.29004406,
        6.73970002, 8.80939022, 7.29555507, 8.16331434, 4.72802467,
        8.8228175 , 8.42349249, 6.06103417, 8.6021218 , 9.00982577,
        7.48259721, 6.29011721, 5.12941313, 8.0756545 , 6.76958372,
        8.31052428, 7.58971817, 8.72718993, 4.89139767, 8.45385098,
        6.36349815, 5.41137928, 6.56629816, 7.06

In [286]:
import os

path = './predict/th'
os.makedirs(path, exist_ok=True)

for i, key in enumerate(grids_th.keys()):
  result_th = pd.merge(private_df_th['username'].reset_index(drop=True), pd.DataFrame(y_pred_th[i], columns=['predict']), left_index=True, right_index=True, how='left')
  file_path = f'./predict/th/{key}_th.csv'
  result_th.to_csv(file_path, header=False, index=False)    

### TBTL

In [287]:
y_pred_tbtl = []
X_private_tbtl = private_df_tbtl.iloc[:, 0:-2].values
for i in grids_tbtl.keys():
    y_pred_tbtl.append(grids_tbtl[i].predict(X_private_tbtl))

In [288]:
y_pred_tbtl

[array([7.10227981, 8.33757965, 7.88397045, 7.85076421, 7.71614775,
        7.66383023, 8.33099438, 8.55267085, 8.19538298, 7.9877563 ,
        7.83987217, 7.96803525, 7.95409543, 8.44090013, 7.99304348,
        8.64876054, 7.5180554 , 7.97619072, 8.46183675, 7.83184204,
        8.7149299 , 8.26764341, 7.84054242, 7.98785514, 7.25243745,
        7.85455866, 7.56145484, 7.60691665, 8.46162397, 7.88616984,
        7.7042018 , 7.68128834, 7.78186035, 7.47561429, 7.56676588,
        8.17207671, 7.56548184, 8.0381387 , 8.64672231, 8.55582366,
        8.00462927, 7.73492955, 8.17154328, 8.28296035, 7.70290193,
        8.17048745, 7.5580844 , 7.50736588, 8.40039953, 7.55211121,
        7.52520909, 8.00762486, 7.90397401, 8.66371973, 7.15987078,
        7.75434584, 7.99424327, 8.63318527, 7.64675157, 7.75812312,
        8.32421986, 8.01926027, 7.51999792, 7.81289167, 8.31370828,
        8.34295112, 7.91393281, 8.3276788 , 6.92443023, 8.49106433,
        8.1265404 , 8.37168439, 8.0277361 , 7.38

In [289]:
import os

path = './predict/tbtl'
os.makedirs(path, exist_ok=True)


for i, key in enumerate(grids_tbtl.keys()):
  result_tbtl = pd.merge(private_df_tbtl['username'].reset_index(drop=True), pd.DataFrame(y_pred_tbtl[i], columns=['predict']), left_index=True, right_index=True, how='left')
  file_path = f'./predict/tbtl/{key}_tbtl.csv'
  result_tbtl.to_csv(file_path, header=False, index=False)    


### CK

In [290]:
y_pred_ck = []
X_private_ck = private_df_ck.iloc[:, 0:-3].values
for i in grids_ck.keys():
    y_pred_ck.append(grids_ck[i].predict(X_private_ck))

In [291]:
y_pred_ck

[array([4.10645792, 3.7745518 , 4.69383885, 6.77375234, 4.57724392,
        4.7436825 , 3.25159567, 4.02874524, 4.5062425 , 4.18404369,
        4.08565188, 4.36715372, 3.68736854, 2.92981627, 2.08168337,
        4.9262528 , 2.95328001, 4.43794286, 4.41231692, 3.89236964,
        4.1560316 , 5.20667524, 3.43090133, 4.59127139, 5.00488548,
        3.94707258, 3.72033777, 4.58917136, 3.0913285 , 6.56231977,
        4.62374527, 3.22100985, 2.71805296, 4.7332537 , 3.98116865,
        4.97662109, 4.7176337 , 4.8651017 , 1.67138983, 3.04613463,
        3.2582676 , 4.57990375, 5.08099055, 5.28575761, 6.77600873,
        7.15175494, 6.35516881, 5.23056306, 5.84763   , 6.1856088 ,
        4.81449841, 6.65112359, 6.6448127 , 9.32447107, 2.7093511 ,
        5.38326815, 8.78071926, 4.31080758, 7.29648337, 5.72067215,
        6.27612561, 6.06096815, 4.51470879, 5.78440389, 4.69913222,
        4.00851706, 5.85367922, 4.43741634, 4.26964674, 4.87081439,
        4.49550711, 4.62016151, 5.5508969 , 4.87

In [292]:
import os

path = './predict/ck'
os.makedirs(path, exist_ok=True)

for i, key in enumerate(grids_ck.keys()):
  result_ck = pd.merge(private_df_ck['username'].reset_index(drop = True), pd.DataFrame(y_pred_ck[i], columns=['predict']), left_index=True, right_index=True, how='left')
  file_path = f'./predict/ck/{key}_ck.csv'
  result_ck.to_csv(file_path, header=False, index=False)  


# 8. Đánh giá lại tầm quan trọng của feature

## 8.1. QT

In [293]:
feature_importance_qt = {}

for name, model in grids_qt.items():
    try:
        importances = grids_qt[model_name].best_estimator_.feature_importances_
    except:
        continue
    feature_importance_df = pd.DataFrame({'Feature': public_df_qt.columns[0:-3], 'Importance': importances}).sort_values(by='Importance', ascending=False).reset_index(drop=True)
    feature_importance_qt[name] = feature_importance_df

for name, df_importance in feature_importance_qt.items():
    print(name)
    print(df_importance)

RandomForest
                                Feature  Importance
0                  assignment_encode_qt    0.551742
1                     problem_encode_qt    0.032495
2               difficult_finished_rate    0.025178
3                    avg_pre_score_easy    0.018760
4   average_solved_submissions_per_week    0.017330
..                                  ...         ...
74                               week 8    0.000000
75                              week 17    0.000000
76                              week 15    0.000000
77                              week 14    0.000000
78                              week 37    0.000000

[79 rows x 2 columns]
XGBoost
                                Feature  Importance
0                  assignment_encode_qt    0.551742
1                     problem_encode_qt    0.032495
2               difficult_finished_rate    0.025178
3                    avg_pre_score_easy    0.018760
4   average_solved_submissions_per_week    0.017330
..                  

## 8.2. TH

In [294]:
feature_importance_th = {}

for model_name in grids_th.keys():
    try:
        importances = grids_th[model_name].best_estimator_.feature_importances_
    except:
        continue
    feature_importance_df = pd.DataFrame({'Feature': public_df_th.columns[0:-3], 'Importance': importances}).sort_values(by='Importance', ascending=False).reset_index(drop=True)
    feature_importance_th[model_name] = feature_importance_df

for name, df_importance in feature_importance_th.items():
    print(name)
    print(df_importance.to_string())

RandomForest
                                      Feature    Importance
0                           problem_encode_th  2.373181e-01
1                          easy_finished_rate  8.729900e-02
2                        assignment_encode_th  6.597807e-02
3                        avg_pre_score_medium  3.608934e-02
4         average_solved_submissions_per_week  3.428211e-02
5                              total_problems  3.074779e-02
6                                      week 2  2.446782e-02
7                     avg_pre_score_difficult  2.267566e-02
8                          avg_pre_score_easy  2.067376e-02
9                    avg_improve_score_medium  1.853933e-02
10                avg_improve_score_difficult  1.826695e-02
11                             difficult_rate  1.813354e-02
12                                  Wrong_avg  1.698899e-02
13                                  easy_rate  1.678977e-02
14                                      Wrong  1.559065e-02
15                  avg_sub

## 8.3. TBTL

In [295]:
feature_importance_tbtl = {}

for name in grids_tbtl.keys():
    try:
        importances = grids_tbtl[name].best_estimator_.feature_importances_
    except:
        continue
    feature_importance_df = pd.DataFrame({'Feature': public_df_tbtl.columns[0:-2], 'Importance': importances}).sort_values(by='Importance', ascending=False).reset_index(drop=True)
    feature_importance_tbtl[name] = feature_importance_df

for name, df_importance in feature_importance_tbtl.items():
    print(name)
    print(df_importance.to_string())

RandomForest
                                      Feature  Importance
0                         problem_encode_tbtl    0.268765
1                      assignment_encode_tbtl    0.094676
2                        medium_finished_rate    0.037459
3                          rate_early_correct    0.030888
4                      avg_improve_score_easy    0.021641
5                        avg_pre_score_medium    0.020836
6                                 medium_rate    0.020786
7                                     week 48    0.020577
8                 avg_improve_score_difficult    0.019704
9                     avg_pre_score_difficult    0.019135
10                   avg_improve_score_medium    0.016887
11                  avg_submissions_difficult    0.016827
12               avg_max_improve_score_medium    0.016282
13                         avg_pre_score_easy    0.015425
14                 average_submissions_medium    0.015253
15                             difficult_rate    0.014446
1

## 8.4. CK

In [296]:
feature_importance_ck = {}

for name in grids_ck.keys():
    try:
        importances = grids_ck[name].best_estimator_.feature_importances_
    except:
        continue
    feature_importance_df = pd.DataFrame({'Feature': public_df_ck.columns[0:-3], 'Importance': importances}).sort_values(by='Importance', ascending=False).reset_index(drop=True)
    feature_importance_ck[name] = feature_importance_df

for name, df_importance in feature_importance_ck.items():
    print(name)
    print(df_importance.to_string())

RandomForest
                                      Feature  Importance
0                           problem_encode_ck    0.279122
1                        assignment_encode_ck    0.106584
2                        avg_pre_score_medium    0.037957
3         average_solved_submissions_per_week    0.021535
4                                       Wrong    0.021435
5                  avg_max_improve_score_easy    0.020318
6                                      week 2    0.020161
7                 avg_improve_score_difficult    0.019660
8                              difficult_rate    0.019128
9                                     week 36    0.018393
10                     avg_improve_score_easy    0.017376
11                    avg_pre_score_difficult    0.017148
12                  avg_submissions_difficult    0.016578
13                                medium_rate    0.016223
14                         Limit_Exceeded_avg    0.015342
15               avg_max_improve_score_medium    0.015334
1

# Lưu thông tin về đặc trưng thời gian chạy và điểm r2-score

In [ ]:
import os

path = './model'
os.makedirs(path, exist_ok=True) 

## QT

In [1]:
columns = ['n_features','features', 'elapsed_time', 'r2_score']
df_info_qt = pd.DataFrame(columns=columns)

NameError: name 'pd' is not defined

In [315]:
columns_qt = full_df_temp_qt.columns.tolist()
elapsed_times_qt = []
for model_name, elapsed_time in grid_search_times_qt.items():
    elapsed_times_qt.append(f"{model_name}: {elapsed_time}")

r2_scores_qt = []
for model_name, model in grids_qt.items():
    r2_scores_qt.append(f"{model_name}: {model.best_score_}")

print(columns_qt)
print(elapsed_times_qt)
print(r2_scores_qt)

['username', 'total_problems', 'total_assignments', 'rate_early_correct', 'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg', 'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error', 'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg', 'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate', 'easy_finished_rate', 'medium_finished_rate', 'avg_submissions_per_assignment', 'average_solved_submissions_per_week', 'avg_submissions_per_week', 'avg_submissions_difficult', 'avg_submissions_easy', 'average_submissions_medium', 'week 1', 'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8', 'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14', 'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20', 'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40', 'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46', 'week 47', 'week 48', 'week 49', 'week 50', 'week 51', 'week 52', 'assignment_encode_qt', '

In [316]:
df_info_qt.loc[len(df_info_qt)] = [len(columns_qt), columns_qt, elapsed_times_qt, r2_scores_qt]

In [317]:
df_info_qt

,n_features,features,elapsed_time,r2_score
0,80,"[username, total_problems, total_assignments, ...","[RandomForest: 673.8085942268372, XGBoost: 72....","[RandomForest: 0.5186923533378405, XGBoost: 0...."


In [319]:
file_path_qt = path + '/info_qt.csv'
df_info_qt.to_csv(file_path_qt)

## TH

In [ ]:
columns = ['n_features','features', 'elapsed_time', 'r2_score']
df_info_th = pd.DataFrame(columns=columns)

NameError: name 'pd' is not defined

In [ ]:
columns_th = full_df_temp_th.columns.tolist()
elapsed_times_th = []
for model_name, elapsed_time in grid_search_times_th.items():
    elapsed_times_th.append(f"{model_name}: {elapsed_time}")

r2_scores_th = []
for model_name, model in grids_th.items():
    r2_scores_th.append(f"{model_name}: {model.best_score_}")

print(columns_th)
print(elapsed_times_th)
print(r2_scores_th)

['username', 'total_problems', 'total_assignments', 'rate_early_correct', 'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg', 'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error', 'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg', 'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate', 'easy_finished_rate', 'medium_finished_rate', 'avg_submissions_per_assignment', 'average_solved_submissions_per_week', 'avg_submissions_per_week', 'avg_submissions_difficult', 'avg_submissions_easy', 'average_submissions_medium', 'week 1', 'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8', 'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14', 'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20', 'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40', 'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46', 'week 47', 'week 48', 'week 49', 'week 50', 'week 51', 'week 52', 'assignment_encode_qt', '

In [ ]:
df_info_th.loc[len(df_info_th)] = [len(columns_th), columns_th, elapsed_times_th, r2_scores_th]

In [ ]:
df_info_th

,n_features,features,elapsed_time,r2_score
0,80,"[username, total_problems, total_assignments, ...","[RandomForest: 673.8085942268372, XGBoost: 72....","[RandomForest: 0.5186923533378405, XGBoost: 0...."


In [ ]:
import os

path = './model'
os.makedirs(path, exist_ok=True) 
file_path_th = path + '/info_th.csv'
df_info_th.to_csv(file_path_th)

## TBTL

In [ ]:
columns = ['n_features','features', 'elapsed_time', 'r2_score']
df_info_tbtl = pd.DataFrame(columns=columns)

NameError: name 'pd' is not defined

In [ ]:
columns_tbtl = full_df_temp_tbtl.columns.tolist()
elapsed_times_tbtl = []
for model_name, elapsed_time in grid_search_times_tbtl.items():
    elapsed_times_tbtl.append(f"{model_name}: {elapsed_time}")

r2_scores_tbtl = []
for model_name, model in grids_tbtl.items():
    r2_scores_th.append(f"{model_name}: {model.best_score_}")

print(columns_tbtl)
print(elapsed_times_tbtl)
print(r2_scores_tbtl)

['username', 'total_problems', 'total_assignments', 'rate_early_correct', 'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg', 'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error', 'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg', 'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate', 'easy_finished_rate', 'medium_finished_rate', 'avg_submissions_per_assignment', 'average_solved_submissions_per_week', 'avg_submissions_per_week', 'avg_submissions_difficult', 'avg_submissions_easy', 'average_submissions_medium', 'week 1', 'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8', 'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14', 'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20', 'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40', 'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46', 'week 47', 'week 48', 'week 49', 'week 50', 'week 51', 'week 52', 'assignment_encode_qt', '

In [ ]:
df_info_tbtl.loc[len(df_info_tbtl)] = [len(columns_tbtl), columns_tbtl, elapsed_times_tbtl, r2_scores_tbtl]

In [ ]:
df_info_tbtl

,n_features,features,elapsed_time,r2_score
0,80,"[username, total_problems, total_assignments, ...","[RandomForest: 673.8085942268372, XGBoost: 72....","[RandomForest: 0.5186923533378405, XGBoost: 0...."


In [ ]:
import os

path = './model'
os.makedirs(path, exist_ok=True) 
file_path_tbtl = path + '/info_tbtl.csv'
df_info_tbtl.to_csv(file_path_tbtl)

## CK

In [ ]:
columns = ['n_features','features', 'elapsed_time', 'r2_score']
df_info_ck = pd.DataFrame(columns=columns)

NameError: name 'pd' is not defined

In [ ]:
columns_ck = full_df_temp_ck.columns.tolist()
elapsed_times_ck = []
for model_name, elapsed_time in grid_search_times_ck.items():
    elapsed_times_ck.append(f"{model_name}: {elapsed_time}")

r2_scores_ck = []
for model_name, model in grids_ck.items():
    r2_scores_ck.append(f"{model_name}: {model.best_score_}")

print(columns_ck)
print(elapsed_times_ck)
print(r2_scores_ck)

['username', 'total_problems', 'total_assignments', 'rate_early_correct', 'not_completed', 'sucessful_rate_avg', 'unsucessful_rate_avg', 'Limit_Exceeded', 'Limit_Exceeded_avg', 'Run_Time_Error', 'Run_Time_Error_avg', 'Wrong', 'Wrong_avg', 'Banned', 'Banned_avg', 'difficult_rate', 'easy_rate', 'medium_rate', 'difficult_finished_rate', 'easy_finished_rate', 'medium_finished_rate', 'avg_submissions_per_assignment', 'average_solved_submissions_per_week', 'avg_submissions_per_week', 'avg_submissions_difficult', 'avg_submissions_easy', 'average_submissions_medium', 'week 1', 'week 2', 'week 3', 'week 4', 'week 5', 'week 6', 'week 7', 'week 8', 'week 9', 'week 10', 'week 11', 'week 12', 'week 13', 'week 14', 'week 15', 'week 16', 'week 17', 'week 18', 'week 19', 'week 20', 'week 21', 'week 36', 'week 37', 'week 38', 'week 39', 'week 40', 'week 41', 'week 42', 'week 43', 'week 44', 'week 45', 'week 46', 'week 47', 'week 48', 'week 49', 'week 50', 'week 51', 'week 52', 'assignment_encode_qt', '

In [ ]:
df_info_ck.loc[len(df_info_ck)] = [len(columns_ck), columns_ck, elapsed_times_ck, r2_scores_ck]

In [ ]:
df_info_ck

,n_features,features,elapsed_time,r2_score
0,80,"[username, total_problems, total_assignments, ...","[RandomForest: 673.8085942268372, XGBoost: 72....","[RandomForest: 0.5186923533378405, XGBoost: 0...."


In [ ]:
import os

path = './model'
os.makedirs(path, exist_ok=True) 
file_path_ck = path + '/info_ck.csv'
df_info_ck.to_csv(file_path_ck)